ライブラリインポート

In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
import matplotlib.ticker as ptick 
from mpl_toolkits.mplot3d import axes3d

import nbimporter
import Weather
import SolarPosition
import HeatingCoolingPeriod
import DivisionDiffuseRatio
import SplittingGlobal2BeamDiffuseRadiation

Importing Jupyter notebook from Weather.ipynb
Importing Jupyter notebook from SolarPosition.ipynb
Importing Jupyter notebook from HeatingCoolingPeriod.ipynb
Importing Jupyter notebook from DivisionDiffuseRatio.ipynb
Importing Jupyter notebook from SplittingGlobal2BeamDiffuseRadiation.ipynb


# 開口部の評価方法に関する仕様書

- LV2(#2) 計算方法名  
    - LV3(#3) 入力値  
    - LV3(#3) 出力値  
    - LV3(#3) 計算方法  
        - LV4(#4) 各計算方法  
    - LV3(#3) 計算例

　本資料は開口部における熱貫流率および日射熱取得率を計算する方法についてまとめたものです。本資料は計算方法を示すとともに、プログラムのテストの入出力についてもまとめています。  
　本資料は以下のパートから構成されています。  
  
[1. 気象データの読込](#1.-%E6%B0%97%E8%B1%A1%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E8%AA%AD%E8%BE%BC)  
　[1.1. 外気温と水平面全天日射量](#1.1.-%E5%A4%96%E6%B0%97%E6%B8%A9%E3%81%A8%E6%B0%B4%E5%B9%B3%E9%9D%A2%E5%85%A8%E5%A4%A9%E6%97%A5%E5%B0%84%E9%87%8F)  
　[1.2. 計算結果確認用気象データ](#1.2.-%E8%A8%88%E7%AE%97%E7%B5%90%E6%9E%9C%E7%A2%BA%E8%AA%8D%E7%94%A8%E6%B0%97%E8%B1%A1%E3%83%87%E3%83%BC%E3%82%BF)  

[2. 暖房期間と冷房期間の計算](#2.-%E6%9A%96%E6%88%BF%E6%9C%9F%E9%96%93%E3%81%A8%E5%86%B7%E6%88%BF%E6%9C%9F%E9%96%93%E3%81%AE%E8%A8%88%E7%AE%97)  

[3. 開口面日射量の計算](#3.-%E9%96%8B%E5%8F%A3%E9%9D%A2%E6%97%A5%E5%B0%84%E9%87%8F%E3%81%AE%E8%A8%88%E7%AE%97)  
　[3.1. 太陽位置](#3.1.-%E5%A4%AA%E9%99%BD%E4%BD%8D%E7%BD%AE)  
　[3.2. 拡散日射の微小要素分割](#3.2.-%E6%8B%A1%E6%95%A3%E6%97%A5%E5%B0%84%E3%81%AE%E5%BE%AE%E5%B0%8F%E8%A6%81%E7%B4%A0%E5%88%86%E5%89%B2)  
　　[3.2.1. 天空日射](#3.2.1.-%E5%A4%A9%E7%A9%BA%E6%97%A5%E5%B0%84)  
　　[3.2.2. 地表面反射日射](#3.2.2.-%E5%9C%B0%E8%A1%A8%E9%9D%A2%E5%8F%8D%E5%B0%84%E6%97%A5%E5%B0%84)  
　[3.3. 直散分離](#3.3.-%E7%9B%B4%E6%95%A3%E5%88%86%E9%9B%A2)  
　[3.4. 斜面（開口面）日射量](#3.4.-%E6%96%9C%E9%9D%A2%EF%BC%88%E9%96%8B%E5%8F%A3%E9%9D%A2%EF%BC%89%E6%97%A5%E5%B0%84%E9%87%8F)  

[4. ひさし等の日除けの計算](#4.-%E3%81%B2%E3%81%95%E3%81%97%E7%AD%89%E3%81%AE%E6%97%A5%E9%99%A4%E3%81%91%E3%81%AE%E8%A8%88%E7%AE%97)  

[5. 窓の日射熱取得率の計算](#5.-%E7%AA%93%E3%81%AE%E6%97%A5%E5%B0%84%E7%86%B1%E5%8F%96%E5%BE%97%E7%8E%87%E3%81%AE%E8%A8%88%E7%AE%97)  
　5.1. グレージング複合体の日射熱取得率  
　　5.1.1. 面材の光学特性  
　　　5.1.1.1. ガラス  
　　　5.1.1.2. ロールスクリーン  
　　　5.1.1.3. ブラインド※  
　　　　5.1.1.3.1. 幾何学形状の作成※  
　　　　5.1.1.3.2. 鏡面反射成分の計算※  
　　　　5.1.1.3.3. 拡散反射成分の計算※  
　　　　5.1.1.3.4. 全体計算※  
　　5.1.2. 短波長放射（日射）の多重反射  
　　5.1.3. 熱抵抗  
　　　5.1.3.1. 面材の熱抵抗  
　　　5.1.3.2. 中空層の熱抵抗  
　　　5.1.3.3. 表面熱伝達抵抗  
　　　5.1.3.4. 熱平衡  
　　5.1.4. 全体計算  
　　　5.1.4.1. 日射量の単位変換  
　　　　5.1.4.1.1. MJ/(m<sup>2</sup>･h) → W/m<sup>2</sup>  
　　　　5.1.4.1.2. W/m<sup>2</sup> → MJ/(m<sup>2</sup>･h)  
　　　5.1.4.2. 開口面入射日射量  
　　　5.1.4.3. 開口面透過日射量  
　　　5.1.4.4. 各層の吸収日射量  
　　　5.1.4.5. 室内側へ流入する吸収日射量  
　　　5.1.4.6. 日射熱取得率   
　5.2. フレームの日射熱取得率  
　5.3. 全体計算  
　　　5.3.1. 時刻別  
　　　5.3.2. 期間平均  
　　　　5.3.2.1. 暖房期間  
　　　　5.3.2.2. 冷房期間 
6. 窓の熱貫流率の計算※  

※は、将来的に記述するパートです。


## 1. 気象データの読込  

### 1.1. 外気温と水平面全天日射量

[See Weather.ipynb Section 1.1](Weather.ipynb#1.1.-%E5%A4%96%E6%B0%97%E6%B8%A9%E5%BA%A6%E3%81%A8%E6%B0%B4%E5%B9%B3%E9%9D%A2%E5%85%A8%E5%A4%A9%E6%97%A5%E5%B0%84%E9%87%8F)

### 1.2. 計算結果確認用気象データ

[See Weather.ipynb Section 1.2](Weather.ipynb#1.2.-%E8%A8%88%E7%AE%97%E7%B5%90%E6%9E%9C%E7%A2%BA%E8%AA%8D%E7%94%A8%E6%B0%97%E8%B1%A1%E3%83%87%E3%83%BC%E3%82%BF)

## 2. 暖房期間と冷房期間の計算  

[See HeatingCoolingPeriod.ipynb](HeatingCoolingPeriod.ipynb)

## 3. 開口面日射量の計算  

### 3.1. 太陽位置  

[See SolarPosition.ipynb](SolarPosition.ipynb)

### 3.2. 拡散日射の微小要素分割

### 3.2.1. 天空日射  

[See DivisionDiffuseRatio.ipynb Section 1.1](DivisionDiffuseRatio.ipynb#1.1.-%E5%A4%A9%E7%A9%BA%E6%97%A5%E5%B0%84)

### 3.2.2. 地表面反射日射  

[See DivisionDiffuseRatio.ipynb Section 1.2](DivisionDiffuseRatio.ipynb#1.2.-%E5%9C%B0%E8%A1%A8%E9%9D%A2%E5%8F%8D%E5%B0%84%E6%97%A5%E5%B0%84)

### 3.3. 直散分離  

ここでは、Watanabe Modelを用いて計算する。  
[See SplittingGlobal2BeamDiffuseRadiation.ipynb Section 1.1](SplittingGlobal2BeamDiffuseRadiation.ipynb#1.1.-Watanabe-Model)  

### 3.4. 斜面（開口面）日射量  

ここでは、Isotropic Modelを用いて計算する。  
[See SolarRadiationOnTiltedSurface.ipynb Section 1.1](SolarRadiationOnTiltedSurface.ipynb#1.1.-Isotropic-Model)  

## 4. ひさし等の日除けの計算  

## 5. 窓の日射熱取得率の計算 

### 5.1. グレージング複合体の日射熱取得率  

### 5.1.1. 面材の光学特性

### 5.1.1.1. ガラス

説明：垂直入射時のガラスの日射透過率及び日射反射率から任意の入射角におけるガラスの日射透過率及び日射反射率を計算する。

### 入力値  

$ \tau_{0,f} $ ：ガラスの正面側からの入射光に対する垂直入射時の日射透過率  
$ \tau_{0,b} $ ：ガラスの背面側からの入射光に対する垂直入射時の日射透過率  
$ \rho_{0,f} $ ：ガラスの正面側からの入射光に対する垂直入射時の日射反射率  
$ \rho_{0,b} $ ：ガラスの背面側からの入射光に対する垂直入射時の日射反射率  
$ \phi $       ：日射の入射角（°）  
gtype          ：ガラスの種類flag（0：透明フロート板ガラス、1：Low-Eガラス）  
ctypef         ：正面側の膜の有無flag（0：膜なし、1：膜あり）  
ctypeb         ：背面側の膜の有無flag（0：膜なし、1：膜あり）  

### 出力値

$ \tau_{\phi,f} $ ：ガラスの正面側からの入射光に対する入射角$\phi$の日射透過率  
$ \tau_{\phi,b} $ ：ガラスの背面側からの入射光に対する入射角$\phi$の日射透過率  
$ \rho_{\phi,f} $ ：ガラスの正面側からの入射光に対する入射角$\phi$の日射反射率  
$ \rho_{\phi,b} $ ：ガラスの背面側からの入射光に対する入射角$\phi$の日射反射率

### 計算方法

#### データ構造と値の範囲

$\tau_{0,f}$・$\tau_{0,b}$・$\rho_{0,f}$・$\rho_{0,b}$・gtype・ctypef・ctypebの組み合わせを1つの変数（タプル）で表し"glass_input"とする。  
全ての値は0.0以上1.0以下の値をとる。  
$\tau_{0,f}$ と $\rho_{0,f}$ の合計は1.0を超えない。  
$\tau_{0,b}$ と $\rho_{0,b}$ の合計は1.0を超えない。   

In [ ]:
glass_input = namedtuple('glass_input','glass_TRS0f glass_TRS0b glass_REF0f glass_REF0b glass_gtype glass_ctypef glass_ctypeb')
# glass_TRS0f：正面側からの入射光に対する垂直入射時の日射透過率
# glass_TRS0f：正面側からの入射光に対する垂直入射時の日射透過率
# glass_TRS0b：背面側からの入射光に対する垂直入射時の日射透過率
# glass_REF0f：正面側からの入射光に対する垂直入射時の日射反射率
# glass_REF0b：背面側からの入射光に対する垂直入射時の日射反射率
# glass_gtype：ガラスの種類flag（0：透明フロート板ガラス、1：Low-Eガラス）
# glass_ctypef：正面側の膜の有無flag（0：膜なし、1：膜あり）
# glass_ctypeb：背面側の膜の有無flag（0：膜なし、1：膜あり）

#### 入射角$\phi$の日射透過率及び日射反射率の計算  

（参考：JIS A 2103：2014 附属書G 斜入射に対する板ガラスの光学特性の計算方法）  

入射角によって日射特性が変化するガラスの日射透過率及び日射反射率の計算方法を以下に示す。  

$ \begin{align}\tau_{\phi,f} = \tau_{0,f}･\sum_{i=0}^5 m_{i}･\cos^i\phi\end{align} $ ･････････････････････････････ (5.1.1.1-a)

$ \begin{align}\tau_{\phi,b} = \tau_{0,b}･\sum_{i=0}^5 m_{i}･\cos^i\phi\end{align} $ ･････････････････････････････ (5.1.1.1-b)

$ \begin{align}\rho_{\phi,f} =\rho_{0,f}+(1-\rho_{0,f})･\sum_{i=0}^5 m_{i}･\cos^i\phi\end{align} $ ･････････････････ (5.1.1.1-c)

$ \begin{align}\rho_{\phi,b} =\rho_{0,b}+(1-\rho_{0,b})･\sum_{i=0}^5 m_{i}･\cos^i\phi\end{align} $ ･････････････････ (5.1.1.1-d)  

ここで、  

$ m_{i} $ ：ガラス種類ごとの係数  

である。

$ m_{i} $ の係数表を以下に示す（参考：JIS A 2103：2014 表G.1）。なお、本仕様書では透明フロート板ガラス及びLow-Eガラスのみについてプログラムを記述する。  

| 板ガラス品種分類              | 規準化透過率又は規準化反射率 | $m_0$ | $m_1$ | $m_2$ | $m_3$ | $m_4$ | $m_5$ |
| :---                      | :---                         | ---: | ---: | ---: | ---: | ---: | ---: |
| 透明フロート板ガラス            | 規準化透過率                 | 0.000 | 2.552 | 1.364 | -11.388 | 13.617 | -5.146 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.189 | 12.392 | -16.593 | 11.851 | -3.461 |
| 熱線吸収板ガラス              | 規準化透過率                 | 0.000 | 1.760 | 3.770 | -14.901 | 16.422 | -6.052 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.189 | 12.392 | -16.593 | 11.851 | -3.461 |
| 熱線反射ガラス                | 規準化透過率                 | 0.000 | 3.297 | -1.122 | -8.408 | 12.206 | -4.972 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.842 | 15.264 | -21.642 | 15.948 | -4.727 |
|                               | 規準化反射率(膜面)           | 1.000 | -5.842 | 15.264 | -21.642 | 15.948 | -4.727 |
| 熱線吸収熱線反射ガラス        | 規準化透過率                 | 0.000 | 2.558 | 0.989 | -11.214 | 14.275 | -5.608 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.842 | 15.264 | -21.642 | 15.948 | -4.727 |
|                               | 規準化反射率(膜面)           | 1.000 | -5.842 | 15.264 | -21.642 | 15.948 | -4.727 |
| 高性能熱線反射ガラスSシリーズ | 規準化透過率                 | 0.000 | 5.342 | -10.170 | 6.670 | 0.886 | -1.728 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -6.459 | 18.722 | -28.982 | 22.804 | -7.085 |
|                               | 規準化反射率(膜面)           | 1.000 | -9.187 | 33.681 | -59.593 | 50.413 | -16.314 |
| 高性能熱線反射ガラスTシリーズ | 規準化透過率                 | 0.000 | 2.552 | 1.364 | -11.388 | 13.617 | -5.146 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.782 | 14.816 | -20.567 | 14.862 | -4.331 |
|                               | 規準化反射率(膜面)           | 1.000 | -8.900 | 31.953 | -55.934 | 47.050 | -15.176 |
| Low-Eガラス                    | 規準化透過率                 | 0.000 | 2.273 | 1.631 | -10.358 | 11.769 | -4.316 |
|                               | 規準化反射率(ガラス面)       | 1.000 | -5.084 | 12.646 | -18.213 | 13.967 | -4.316 |
|                               | 規準化反射率(膜面)           | 1.000 | -4.387 | 9.175 | -11.152 | 7.416 | -2.052 |

In [ ]:
# 入射角φの日射透過率及び日射反射率の計算
def glass_TRSf(L, phi):
    glass_TRSnf = 0.
    for i in range(0, 6):
        glass_TRSnf += glass_mTRS(L)[i] * math.cos(math.radians(phi)) ** i
    return L.glass_TRS0f * glass_TRSnf

def glass_TRSb(L, phi):
    glass_TRSnb = 0.
    for i in range(0, 6):
        glass_TRSnb += glass_mTRS(L)[i] * math.cos(math.radians(phi)) ** i
    return L.glass_TRS0b * glass_TRSnb
    
def glass_REFf(L, phi):
    glass_REFnf = 0.
    for i in range(0, 6):
        if L.glass_ctypef == 0:
            glass_REFnf += glass_mREFg(L)[i] * math.cos(math.radians(phi)) ** i
        elif L.glass_ctypef == 1:
            glass_REFnf += glass_mREFc(L)[i] * math.cos(math.radians(phi)) ** i
    return L.glass_REF0f + (1 - L.glass_REF0f) * glass_REFnf
    
def glass_REFb(L, phi):
    glass_REFnb = 0.0
    for i in range(0, 6):
        if L.glass_ctypeb == 0:
            glass_REFnb += glass_mREFg(L)[i] * math.cos(math.radians(phi)) ** i
        elif L.glass_ctypeb == 1:
            glass_REFnb += glass_mREFc(L)[i] * math.cos(math.radians(phi)) ** i
    return L.glass_REF0b + (1 - L.glass_REF0b) * glass_REFnb

#係数mの選択。ここでは、透明フロート板ガラス及びLow-Eガラスのみを記述する。
def glass_mTRS(L):
    if L.glass_gtype == 0:    return [0.000, 2.552, 1.364, -11.388, 13.617, -5.146]
    elif L.glass_gtype == 1:  return [0.000, 2.273, 1.631, -10.358, 11.769, -4.316]
    else:                         return ValueError

def glass_mREFg(L):
    if L.glass_gtype == 0:    return [1.000, -5.189, 12.392, -16.593, 11.851, -3.461]
    elif L.glass_gtype == 1:  return [1.000, -5.084, 12.646, -18.213, 13.967, -4.316]
    else:                   return ValueError

def glass_mREFc(L):
    if L.glass_gtype == 0:    return ValueError
    elif L.glass_gtype == 1:  return [1.000, -4.387, 9.175, -11.152, 7.416, -2.052]
    else:                   return ValueError

### 計算例

In [ ]:
glass_result = np.zeros((5,91))
gl_in = glass_input(0.703, 0.703, 0.106, 0.117, 1, 0, 1)
for j in range(0, 91):
    phi = j
    glass_result[0][j] = phi
    glass_result[1][j] = glass_TRSf(gl_in, phi)
    glass_result[2][j] = glass_TRSb(gl_in, phi)
    glass_result[3][j] = glass_REFf(gl_in, phi)
    glass_result[4][j] = glass_REFb(gl_in, phi)
   
np.savetxt('glass_result.csv', np.transpose(glass_result), fmt='%.10f', delimiter=',')

#グラフの描画
graph_num = 2
ax = np.empty(graph_num, dtype = np.object)
fig = plt.figure(figsize = (15, 5 * ((graph_num + 1) // 2)))
plt.style.use('ggplot')

gpc = ['C0', 'C1']
gxd = [[glass_result[0][0:91], glass_result[0][0:91]], [glass_result[0][0:91], glass_result[0][0:91]]]
gyd = [[glass_result[1][0:91], glass_result[2][0:91]], [glass_result[3][0:91], glass_result[4][0:91]]]
gxl = 'φ[°]'
gyl = ['τ (°)', 'ρ (°)']
gxt = [10 * i for i in range(10)]
gyt = [0.1 * i for i in range(11)]
    
for i in range(graph_num):   
    ax[i] = fig.add_subplot((graph_num + 1) // 2, 2, i + 1)
    for j in range(2):
        ax[i].plot(gxd[i][j], gyd[i][j], color = gpc[j])
    ax[i].set_xticks(gxt)
    ax[i].set_yticks(gyt)
    ax[i].set_xlabel(gxl, fontsize = 12)
    ax[i].set_ylabel(gyl[i], fontsize = 12)

plt.show()

### 5.1.1.2. ロールスクリーン

説明：垂直入射時のロールスクリーンの日射透過率及び日射反射率から任意の入射角におけるロールスクリーンの日射透過率及び日射反射率を計算する。

### 入力値  
$ \tau_{0,f} $ ：ロールスクリーンの正面側からの入射光に対する垂直入射時の日射透過率  
$ \tau_{0,b} $ ：ロールスクリーンの背面側からの入射光に対する垂直入射時の日射透過率  
$ \rho_{0,f} $ ：ロールスクリーンの正面側からの入射光に対する垂直入射時の日射反射率  
$ \rho_{0,b} $ ：ロールスクリーンの背面側からの入射光に対する垂直入射時の日射反射率  
$ \phi$        ：日射の入射角（°）

### 出力値
$ \tau_{\phi,f} $ ：ロールスクリーンの正面側からの入射光に対する入射角$\phi$の日射透過率  
$ \tau_{\phi,b} $ ：ロールスクリーンの背面側からの入射光に対する入射角$\phi$の日射透過率  
$ \rho_{\phi,f} $ ：ロールスクリーンの正面側からの入射光に対する入射角$\phi$の日射反射率  
$ \rho_{\phi,b} $ ：ロールスクリーンの背面側からの入射光に対する入射角$\phi$の日射反射率

### 計算方法  

#### データ構造と値の範囲  

$\tau_{0,f}$・$\tau_{0,b}$・$\rho_{0,f}$の組み合わせを1つの変数（タプル）で表し"role_input"とする。  
全ての値は0.0以上1.0以下の値をとる。  
$\tau_{0,f}$ と $\rho_{0,f}$ の合計は1.0を超えない。  
$\tau_{0,b}$ と $\rho_{0,b}$ の合計は1.0を超えない。  

In [ ]:
role_input = namedtuple('role_input','role_TRS0f role_TRS0b role_REF0f role_REF0b')
# role_TRS0f：正面側からの入射光に対する垂直入射時の日射透過率
# role_TRS0b：背面側からの入射光に対する垂直入射時の日射透過率
# role_REF0f：正面側からの入射光に対する垂直入射時の日射反射率
# role_REF0b：背面側からの入射光に対する垂直入射時の日射反射率

#### 入射角φの日射透過率及び日射反射率の計算

入射角によって日射特性が変化しないロールスクリーンの日射透過率及び日射反射率の計算方法を以下に示す。  

$ \begin{align}\tau_{\phi,f} = \tau_{0,f}\end{align}$ ････････････････････････････････････････････ (5.1.1.2-a)

$ \begin{align}\tau_{\phi,b} = \tau_{0,b}\end{align}$ ････････････････････････････････････････････ (5.1.1.2-b)

$ \begin{align}\rho_{\phi,f} =\rho_{0,f}\end{align}$ ････････････････････････････････････････････ (5.1.1.2-c)

$ \begin{align}\rho_{\phi,b} =\rho_{0,b}\end{align}$ ････････････････････････････････････････････ (5.1.1.2-d)  

In [ ]:
# 入射角φの日射透過率及び日射反射率の計算
def role_TRSf(L, phi):
    return L.role_TRS0f

def role_TRSb(L, phi):
    return L.role_TRS0b
    
def role_REFf(L, phi):
    return L.role_REF0f
    
def role_REFb(L, phi):
    return L.role_REF0b

### 計算例

In [ ]:
role_result = np.zeros((5,91))
rl_in = role_input(0.3, 0.3, 0.63, 0.63)

for j in range(0, 91):
    phi = j
    role_result[0][j] = phi
    role_result[1][j] = role_TRSf(rl_in, phi)
    role_result[2][j] = role_TRSb(rl_in, phi)
    role_result[3][j] = role_REFf(rl_in, phi)
    role_result[4][j] = role_REFb(rl_in, phi)
   
np.savetxt('role_result.csv', np.transpose(role_result), fmt='%.10f', delimiter=',')

#グラフの描画
graph_num = 2
ax = np.empty(graph_num, dtype = np.object)
fig = plt.figure(figsize = (15, 5 * ((graph_num + 1) // 2)))
plt.style.use('ggplot')

gpc = ['C0', 'C1']
gxd = [[role_result[0][0:91], role_result[0][0:91]], [role_result[0][0:91], role_result[0][0:91]]]
gyd = [[role_result[1][0:91], role_result[2][0:91]], [role_result[3][0:91], role_result[4][0:91]]]
gxl = 'φ[°]'
gyl = ['τ (°)', 'ρ (°)']
gxt = [10 * i for i in range(10)]
gyt = [0.1 * i for i in range(11)]
    
for i in range(graph_num):   
    ax[i] = fig.add_subplot((graph_num + 1) // 2, 2, i + 1)
    for j in range(2):
        ax[i].plot(gxd[i][j], gyd[i][j], color = gpc[j])
    ax[i].set_xticks(gxt)
    ax[i].set_yticks(gyt)
    ax[i].set_xlabel(gxl, fontsize = 12)
    ax[i].set_ylabel(gyl[i], fontsize = 12)

plt.show()

### 5.1.1.3. ブラインド

### 5.1.1.3.1. 幾何学形状の作成

### 5.1.1.3.2. 鏡面反射成分の計算

### 5.1.1.3.3. 拡散反射成分の計算

### 5.1.1.3.4. 全体計算

### 5.1.2. 短波長放射（日射）の多重反射 

説明：グレージング・日除け・付属部材等の各層において、短波長日射の正面（室外）側からの透過率、背面（室内）側からの透過率、正面側からの反射率、背面側からの反射率が既知の場合に、短波長の多重反射を解いて、グレージング複合体全体の透過率および、各層で吸収される短波長の吸収率を計算する。  
添え字$i$、$j$は層の番号（0～$n$-1）であり、$n$は層の数を表す。

### 入力値  
$ \tau_{f,j} $ ：層$j$の正面側からの入射光に対する日射透過率  
$ \tau_{b,j} $ ：層$j$の背面側からの入射光に対する日射透過率  
$ \rho_{f,j} $ ：層$j$の正面側からの入射光に対する日射反射率  
$ \rho_{b,j} $ ：層$j$の背面側からの入射光に対する日射反射率

### 出力値  

$ \tau_{glz} $ ：グレージング複合体全体の透過率  
$ \alpha_j $ ：層$j$の正面側からの入射光に対する日射吸収率  

### 計算方法  

#### 層$j$のデータ構造と値の範囲

層1つにつき、$\tau_f$・$\tau_b$・$\rho_f$・$\rho_b$の組み合わせを1つの変数（タプル）で表し"layer"とする。  
全ての値は0.0以上1.0以下の値をとる。  
$\tau_f$ と $\rho_f$ の合計は1.0を超えない。  
$\tau_b$ と $\rho_b$ の合計は1.0を超えない。  

In [ ]:
layer_input = namedtuple('layer_input','ly_TRSf ly_TRSb ly_REFf ly_REFb')
# ly_TRSf：正面側からの入射光に対する日射透過率
# ly_TRSf：正面側からの入射光に対する日射透過率
# ly_TRSb：背面側からの入射光に対する日射透過率
# ly_REFf：正面側からの入射光に対する日射反射率
# ly_REFb：背面側からの入射光に対する日射反射率

#### 任意の層から任意の層までの多重反射計算  

（参考：JIS A 2103：2014 5.4.2 日射透過率及び各面材の日射吸収率）

隣り合う層$i$から層$j$までの値の計算方法を以下に示す。  

$ \begin{align}\tau_{f,i,j} = \frac {\tau_{f,i,j-1}･\tau_{f,j}}{1-\rho_{b,i,j-1}･\rho_{f,j}}\end{align}$ ････････････････････････････････ (5.1.2-a)

$ \begin{align}\tau_{b,i,j} = \frac {\tau_{b,j}･\tau_{b,i,j-1}}{1-\rho_{b,i,j-1}･\rho_{f,j}}\end{align}$ ････････････････････････････････ (5.1.2-b)

$ \begin{align}\rho_{f,i,j} = \rho_{f,i,j-1} + \frac {\tau_{f,i,j-1}･\rho_{f,j}･\tau_{b,i,j-1}}{1-\rho_{b,i,j-1}･\rho_{f,j}}\end{align}$ ･････････････････････ (5.1.2-c)

$ \begin{align}\rho_{b,i,j} = \rho_{b,j} + \frac {\tau_{b,j}･\rho_{b,i,j-1}･\tau_{f,j}}{1-\rho_{b,i,j-1}･\rho_{f,j}}\end{align}$ ･･････････････････････････ (5.1.2-d)

ここで、

$ \tau_{f,i,j} $　：　層$i$－$j$間の正面側からの入射光に対する日射透過率  
$ \tau_{b,i,j} $　：　層$i$－$j$間の背面側からの入射光に対する日射透過率  
$ \rho_{f,i,j} $　：　層$i$－$j$間の正面側からの入射光に対する日射反射率  
$ \rho_{b,i,j} $　：　層$i$－$j$間の背面側からの入射光に対する日射反射率  

である。  

任意の層$i$－$j$間の値を求める場合は、層$i+1$, $i+2$,..., $j-2$, $j-1$, $j$というように、式(5.1.2-a)～式(5.1.2-d)を逐次代入して求める。  

ここで、$ i=j $ のときは、

$ \begin{align}\tau_{f,i,j} = \tau_{f,j}\end{align} $ ････････････････････････････････････････････ (5.1.2-e)

$ \begin{align}\tau_{b,i,j} = \tau_{b,j}\end{align} $ ････････････････････････････････････････････ (5.1.2-f)

$ \begin{align}\rho_{f,i,j} = \rho_{f,j}\end{align} $ ････････････････････････････････････････････ (5.1.2-g)

$ \begin{align}\rho_{b,i,j} = \rho_{b,j}\end{align} $ ････････････････････････････････････････････ (5.1.2-h)

である。

In [ ]:
# 層riから層rjの複合した、TRSf TRSb REFf REFb を算出する。
def Sol_Multi_Lay(L, ri, rj):
    
    # riとrjの数値の範囲に関するエラー処理
    if ri <  0:      raise ValueError
    if rj <  0:      raise ValueError
    if ri >  rj:     raise ValueError
    if rj >= len(L): raise ValueError

    if ri == rj:
        return L[rj]
    else:
        SML = Sol_Multi_Lay(L, ri, rj-1) #層1から層j-1の日射特性値
        trsf = (SML.ly_TRSf * L[rj].ly_TRSf) / (1. - SML.ly_REFb * L[rj].ly_REFf)
        trsb = (L[rj].ly_TRSb * SML.ly_TRSb) / (1. - SML.ly_REFb * L[rj].ly_REFf)
        reff = SML.ly_REFf + (SML.ly_TRSf * L[rj].ly_REFf * SML.ly_TRSb) / (1. - SML.ly_REFb * L[rj].ly_REFf)
        refb = L[rj].ly_REFb + (L[rj].ly_TRSb * SML.ly_REFb * L[rj].ly_TRSf) / (1. - SML.ly_REFb * L[rj].ly_REFf)
        return layer_input(trsf, trsb, reff, refb)

#### 各層の日射吸収率

（参考：JIS A 2103：2014 5.4.2 日射透過率及び各面材の日射吸収率）

各層の値の計算方法を以下に示す。  
（JISでは添え字は1始まりであるが、本仕様書では0始まりとしている。） 

$ j = 0 $ かつ $ n = 0 $ の場合

$ \begin{align} \alpha_j = \alpha_{f,j} \end{align} $ ･･････････････････････････････････････････････ (5.1.2-i)

$ j = 0 $ かつ $ n ≠ 0 $ の場合

$ \begin{align} \alpha_j = \alpha_{f,j} + \frac {\tau_{f,0,j}･\rho_{f,j+1,n}}{1-\rho_{b,0,j}･\rho_{f,j+1,n}}･\alpha_{b,j} \end{align} $ ･･････････････････････ (5.1.2-j)

$ j = 1 $～$ n - 2 $ の場合  

$ \begin{align} \alpha_j = \frac {\tau_{f,0,j-1}}{1-\rho_{b,0,j-1}･\rho_{f,j,n}}･\alpha_{f,j} + \frac {\tau_{f,0,j}･\rho_{f,j+1,n}}{1-\rho_{b,0,j}･\rho_{f,j+1,n}}･\alpha_{b,j} \end{align} $ ････ (5.1.2-k)

$ j = n - 1 $ の場合

$ \begin{align} \alpha_j = \frac {\tau_{f,0,j-1}}{1-\rho_{b,0,j-1} \rho_{f,j,n}}･\alpha_{f,j} \end{align} $ ･････････････････････････････ (5.1.2-l)

ここで、

$ j $ ：層の番号  
$ n $ ：層の数  

である。  

また、

$ \begin{align} \alpha_{f,j} = 1 - \tau_{f,j} - \rho_{f,j} \end{align} $ ･･･････････････････････････････････ (5.1.2-m)

$ \begin{align} \alpha_{b,j} = 1 - \tau_{b,j} - \rho_{b,j} \end{align} $ ･･･････････････････････････････････ (5.1.2-n)

である。

In [ ]:
# 層iの日射吸収率を算出する。
def Abs_Multi_Lay(L, rj):
    
    # 層の正面側からの入射光に対する日射吸収率
    def ABSf(M):
        return 1. - M.ly_TRSf - M.ly_REFf
    
    # 層の背面側からの入射光に対する日射吸収率
    def ABSb(M):
        return 1. - M.ly_TRSb - M.ly_REFb
    
    # 層の最背面の番号
    n = len(L) - 1
    
    # 関数リネーム
    SML = Sol_Multi_Lay
    
    # 層jの正面側からの入射光
    def Sol_Rad_f(L, rj):
        if rj == 0:
            return 1.
        else:
            return SML(L, 0, rj - 1).ly_TRSf / (1. - SML(L, 0, rj - 1).ly_REFb * SML(L, rj, n).ly_REFf)
            
    # 層jの背面側からの入射光
    def Sol_Rad_b(L, rj, n):
        if rj == n:
        
            return 0.
        else:
            return SML(L, 0, rj).ly_TRSf * SML(L, rj + 1, n).ly_REFf / ( 1. - SML(L, 0, rj).ly_REFb * SML(L, rj + 1, n).ly_REFf )
        
    # rjの数値の範囲に関するエラー処理
    if rj < 0: raise ValueError
    if rj > n: raise ValueError
    
    return Sol_Rad_f(L, rj) * ABSf(L[rj]) + Sol_Rad_b(L, rj, n) * ABSb(L[rj])

#### グレージング複合体全体の透過率  

グレージング複合体全体の透過率は、定義から、

$ \begin{align} \tau_{glz} = \tau_{f,0,n-1} \end{align} $ ･････････････････････････････････････････ (5.1.2-o)

となる。  

In [ ]:
# グレージング複合体全体の透過率
def Sol_Trs_All(L):
    
    return Sol_Multi_Lay(L, 0, len(L) - 1).ly_TRSf

### 計算例

In [ ]:
ly_in = [layer_input(0.859, 0.859, 0.077, 0.077),layer_input(0.859, 0.859, 0.077, 0.077),layer_input(0.859, 0.859, 0.077, 0.077)]


layer_result = np.zeros(len(ly_in) + 3)
layer_result[1] = Sol_Multi_Lay(ly_in, 0, len(ly_in) - 1).ly_REFf      #グレージング複合体全体の正面側の反射率
layer_result[2] = Sol_Trs_All(ly_in)                                 #グレージング複合体全体の透過率

sum_abs = 0.
for j in range(0, len(ly_in)):
    layer_result[j + 3] = Abs_Multi_Lay(ly_in, j)                    #層jの日射吸収率
    sum_abs += layer_result[j + 3]

#グレージング複合体全体の正面側の反射率・グレージング複合体全体の透過率・層jの日射吸収率の合計
layer_result[0] = layer_result[1] + layer_result[2] + sum_abs

np.savetxt('layer_result.csv', layer_result, fmt='%.10f', delimiter=',')

### 5.1.3. 熱抵抗  

### 5.1.3.1. 面材の熱抵抗

説明：面材の厚さと熱伝導率から面材の熱抵抗を計算する。

### 入力値  
$ d_{gl,l} $ ：面材を構成する材料$l$の厚さ (m)  
$ \lambda_{gl,l} $ ：面材を構成する材料$l$の熱伝導率 [W/(m･K)]     

### 出力値
$ R_{gl} $ ：面材の熱抵抗 (m<sup>2</sup>･K/W)  

### 計算方法  

#### データ構造と値の範囲  

$ d_{gl} $・$ \lambda_{gl} $の組み合わせを1つの変数（タプル）で表し"gl_input"とする。  

In [ ]:
gl_input = namedtuple('gl_input','gl_d gl_lmd')

#gl_d：グレージング層jの厚さ(m)
#gl_lmd：グレージング層jの熱伝導率[W/(m・K)]

#### 面材の熱抵抗の計算  

（参考：JIS R 2103：2014 5.4.3 面材の熱抵抗）  

面材の熱抵抗の計算方法を以下に示す。 

$ \begin{align}R_{gl} = \sum_{l=1}^{L}\frac{d_{gl,l}}{\lambda_{gl,l}}\end{align}$ ････････････････････････････････････････ (5.1.3.1-a)  

ここで、  

$ R_{gl} $ ：面材の熱抵抗 (m<sup>2</sup>･K/W)  
$ d_{gl,l} $ ：面材を構成する材料$l$の厚さ (m)  
$ \lambda_{gl,l} $ ：面材を構成する材料$l$の熱伝導率 [W/(m･K)]   

である。 

In [ ]:
def Glazing_Resistance(L):
    
    gl_R = 0.
    for l in range(len(L.gl_d)):
        gl_R += L.gl_d[l] / L.gl_lmd[l]

    return gl_R

### 計算例

In [ ]:
Rgl_in = [gl_input([0.003, 0.006], [1., 0.5]),
          gl_input([0.005], [1.])]

Rgl_result = np.zeros(len(Rgl_in))
for j in range(len(Rgl_in)):
    Rgl_result[j] = Glazing_Resistance(Rgl_in[j])

print (Rgl_result)

### 5.1.3.2. 中空層の熱抵抗

説明：面材間の表面温度から中空層の熱抵抗を計算する。

### 入力値  
$ \theta_1 $ ：面1の表面温度 (℃)  
$ \theta_2 $ ：面2の表面温度 (℃)  
$ \epsilon_1 $ ：面1の修正放射率  
$ \epsilon_2 $ ：面2の修正放射率  
$ s $ ：中空層の厚さ (m)  
gap_type_1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gap_type_2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gap_ratio_1：気体1の容積割合（0.0～1.0）  
gap_ratio_2：気体2の容積割合（1.0 - gas_ratio_1）  
gap_dir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）    

### 出力値
$ R_{s} $ ：中空層の熱抵抗 (m<sup>2</sup>･K/W)  

### 計算方法  

#### データ構造と値の範囲  

$ \theta_1 $・$ \theta_2 $・$ \epsilon_1 $・$ \epsilon_2 $・$ s $・gap_type_1・gap_type_2・gap_ratio_1・gap_dirの組み合わせを1つの変数（タプル）で表し"gap_input"とする。  

In [ ]:
gap_input = namedtuple('gap_input','gap_Tmc1 gap_Tmc2 gap_eps1 gap_eps2 gap_s gap_type1 gap_type2 gap_ratio1 gap_dir')

# gap_Tmc1：面1の表面温度（℃）
# gap_Tmc2：面2の表面温度（℃）
# gap_eps1：面1の修正放射率
# gap_eps2：面2の修正放射率
# gap_s：中空層の厚さ（m）
# gap_type1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
# gap_type2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
# gap_ratio1：気体1の容積割合（0.0～1.0）
# gap_dir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）

#### 中空層の熱抵抗の計算  

（参考：JIS R 3107：1998 4.1 基礎式）  

中空層の熱抵抗の計算方法を以下に示す。 

$ \begin{align}R_{s} = \frac{1}{h_r+h_g}\end{align}$ ････････････････････････････････････････ (5.1.3.2-a)  

ここで、  

$ R_{s} $ ：中空層の熱抵抗 (m<sup>2</sup>･K/W)  
$ h_{r} $ ：中空層における放射熱コンダクタンス [W/(m<sup>2</sup>･K)]  
$ h_{g} $ ：中空層における気体の伝導と対流による熱コンダクタンス（気体熱コンダクタンス）[W/(m<sup>2</sup>･K)]  

である。 

#### 中空層の放射熱コンダクタンスの計算  

（参考：JIS R 3107：1998 4.2 放射熱コンダクタンス）  

中空層における放射熱コンダクタンスの計算方法を以下に示す。

$ \begin{align}h_r = 4･\sigma･(\frac{1}{\epsilon_1}+\frac{1}{\epsilon_2}-1)^{-1}･T_m^3\end{align}$ ･･･････････････････････ (5.1.3.2-b)  


ここで、  

$ \sigma $ ：ステファン・ボルツマン定数（＝5.67×10<sup>-7</sup>）[W/(m<sup>2</sup>･K<sup>4</sup>)]  
$ \epsilon_1 $，$ \epsilon_2 $ ：中空層に接する二つのガラス面の修正放射率  
$ T_m $ ：中空層に接する二つのガラス面の絶対温度の平均値 (K)  

である。  

#### 中空層の気体熱コンダクタンスの計算  

（参考：JIS R 3107：1998 4.3 気体熱コンダクタンス）  

中空層における気体熱コンダクタンスの計算方法を以下に示す。

$ \begin{align}h_g = Nu･\frac{\lambda}{s}\end{align}$ ･･････････････････････････････････････････ (5.1.3.2-c)

$ \begin{align}Nu = A･(Gr･Pr)^n\end{align}$ ････････････････････････････････････ (5.1.3.2-d)

$ \begin{align}Gr = \frac{9.81･s^3･\Delta T･\rho^2}{T'_m･\mu^2}\end{align}$ ････････････････････････････････ (5.1.3.2-e)

$ \begin{align}Pr = \frac{\mu･c}{\lambda}\end{align}$ ････････････････････････････････････････････ (5.1.3.2-f)  

ここで、  

$ Nu $ ：ヌセルト数  
$ \lambda $ ：気体の熱伝導率 [W/(m･K)]  
$ s $ ：中空層の厚さ (m)  
$ A $，$ n $ ：中空層の傾斜角と熱流方向によって決まる数値  
$ Gr $ ：グラスホフ数  
$ Pr $ ：プラントル数  
$ \Delta T $ ：中空層に接する二つのガラス面の温度差 (K)  
$ \rho $ ：気体の密度 (kg/m<sup>3</sup>)  
$ T'_m $ ：中空層の気体の平均絶対温度 (K)  
$ \mu $ ：気体の粘度 [kg/(m･s)]  
$ c $ ：気体の比熱 [J/(kg･K)]  

である。

中空層の傾斜角と熱流方向によって決まる数値$ A $，$ n $は、以下の値とする（参考：JIS R3107：1998 5.3.1）。

| 条件 | $A$ | $n$ |
| :--- | ---: | ---: |
| 中空層が垂直で熱流方向が水平の場合 | 0.035 | 0.38 |
| 中空層が水平で熱流方向が上向きの場合 | 0.16 | 0.28 |
| 中空層が45°で熱流方向が上向きの場合 | 0.10 | 0.31 |

また、混合気体の場合には、気体の物性値は、式(5.1.3-g)によって計算する（参考：JIS R3107：1998 5.3.2）。

$ \begin{align}F = \sum_{i=1}^N F_{i}･r_{i}\end{align} $ ･････････････････････････････････････････ (5.1.3.2-g)  

ここで、  

$ F $ ：混合気体の密度、粘度、熱伝導率又は比熱  
$ F_{i} $ ：各気体の密度、粘度、熱伝導率又は比熱  
$ r_{i} $ ：各気体の容積割合  
$ N $ ：気体の数  

である。  
なお、本仕様書では、$N$は1または2とする。

気体の物性値を以下に示す（参考：JIS R 3107：1998 付表2）。  

| 気体 | 温度 $\theta$ (℃) | 密度 $\rho$ (kg/m<sup>3</sup>) | 粘度 $\mu$ [kg/(m･s)] | 熱伝導率 $\lambda$ [W/(m･K)] | 比熱 $c$ [J/(kg･K)] |
| :--- | ---: | ---: | ---: | ---: | ---: |
| 空気 | -10.0 | 1.326 | 1.661e-5 | 2.336e-2 | 1.008e+3 |
|      | 0.0   | 1.277 | 1.711e-5 | 2.416e-2 | 1.008e+3 |
|      | 10.0  | 1.232 | 1.761e-5 | 2.496e-2 | 1.008e+3 |
|      | 20.0  | 1.189 | 1.811e-5 | 2.576e-2 | 1.008e+3 |
| アルゴン | -10.0 | 1.829 | 2.038e-5 | 1.584e-2 | 0.519e+3 |
|          | 0.0   | 1.762 | 2.101e-5 | 1.634e-2 | 0.519e+3 |
|          | 10.0  | 1.699 | 2.164e-5 | 1.684e-2 | 0.519e+3 |
|          | 20.0  | 1.640 | 2.228e-5 | 1.734e-2 | 0.519e+3 |
| SF<sub>6</sub> | -10.0 | 6.844 | 1.383e-5 | 1.119e-2 | 0.614e+3 |
|                | 0.0   | 6.602 | 1.421e-5 | 1.197e-2 | 0.614e+3 |
|                | 10.0  | 6.360 | 1.459e-5 | 1.275e-2 | 0.614e+3 |
|                | 20.0  | 6.118 | 1.497e-5 | 1.354e-2 | 0.614e+3 |
| クリプトン | -10.0 | 3.832 | 2.260e-5 | 0.842e-2 | 0.245e+3 |
|            | 0.0   | 3.690 | 2.330e-5 | 0.870e-2 | 0.245e+3 |
|            | 10.0  | 3.560 | 2.400e-5 | 0.900e-2 | 0.245e+3 |
|            | 20.0  | 3.430 | 2.470e-5 | 0.926e-2 | 0.245e+3 |

In [ ]:
def Gap_Resistance(L):
    gap_ratio2 = 1. - L.gap_ratio1
    gap_dT = abs(L.gap_Tmc1 - L.gap_Tmc2)
    gap_Tmc = (L.gap_Tmc1 + L.gap_Tmc2) / 2.
    gap_Tm = gap_Tmc + 273.15
    ave_Tm = gap_Tm
    
    if L.gap_dir == 0:
        gap_A = 0.035
        gap_n = 0.38
    elif L.gap_dir == 1:
        gap_A = 0.16
        gap_n = 0.28
    elif L.gap_dir == 2:
        gap_A = 0.1
        gap_n = 0.31
    
    sgm = 5.67 * 10. ** (-8.)
    
    for j in range(1, 5):
        gas_mix = gas_prop(L.gap_type1, gap_Tmc, j) * L.gap_ratio1 + gas_prop(L.gap_type2, gap_Tmc, j) * gap_ratio2
        if j == 1: gas_rho = gas_mix
        elif j == 2: gas_mu = gas_mix * 10. ** (-5.)
        elif j == 3: gas_lmd = gas_mix * 10. ** (-2.)
        elif j == 4: gas_c = gas_mix * 10. ** 3.
    gas_Gr = 9.81 * L.gap_s ** 3. * gap_dT * gas_rho ** 2. / (gap_Tm * gas_mu ** 2.)
    gas_Pr = gas_mu * gas_c / gas_lmd
    gas_Nu = gap_A * (gas_Gr * gas_Pr)** gap_n
    if gas_Nu <= 1.: gas_Nu = 1.
    gap_hg = gas_Nu * gas_lmd / L.gap_s
    gap_hr = 4. * sgm * (1. / L.gap_eps1 + 1. / L.gap_eps2 - 1.) ** (-1.) * ave_Tm ** 3.
    return 1. / (gap_hr + gap_hg)

def gas_prop(gap_type, gap_Tmc, j):
    if gap_type == 0: #空気
        gp = np.array([[-10.0, 1.326, 1.661, 2.336, 1.008], [0.0, 1.277, 1.711, 2.416, 1.008],
                       [10.0, 1.232, 1.761, 2.496, 1.008], [20.0, 1.189, 1.811, 2.576, 1.008]])
    
    elif gap_type == 1: #アルゴン
        gp = np.array([[-10.0, 1.829, 2.038, 1.584, 0.519], [0.0, 1.762, 2.101, 1.634, 0.519],
                       [10.0, 1.699, 2.164, 1.684, 0.519], [20.0, 1.640, 2.228, 1.734, 0.519]])

    elif gap_type == 2: #SF6
        gp = np.array([[-10.0, 6.844, 1.383, 1.119, 0.614], [0.0, 6.602, 1.421, 1.197, 0.614],
                       [10.0, 6.360, 1.459, 1.275, 0.614], [20.0, 6.118, 1.497, 1.354, 0.614]])        
 
    elif gap_type == 3: #クリプトン
        gp = np.array([[-10.0, 3.832, 2.260, 0.842, 0.245], [0.0, 3.690, 2.330, 0.870, 0.245],
                       [10.0, 3.560, 2.400, 0.900, 0.245], [20.0, 3.430, 2.470, 0.926, 0.245]])         
    
    for i in range(1, 4):
        if i == 1 and gap_Tmc < gp[i-1, 0]: break
        elif gp[i-1, 0] <= gap_Tmc and gap_Tmc < gp[i, 0]: break
        elif i == 3 and gp[i, 0] <= gap_Tmc: break
       
    gp_cal = gp[i - 1, j] + (gp[i, j] - gp[i - 1, j]) * (gap_Tmc - gp[i - 1, 0]) / (gp[i, 0] - gp[i - 1, 0])
    
    return gp_cal

### 計算例

In [ ]:
Rgap_in = [gap_input(0., 20., 0.837, 0.837, 0.012, 0, 0, 1., 0),
           gap_input(30., 25., 0.837, 0.837, 0.012, 0, 0, 1., 0),
           gap_input(0., 20., 0.837, 0.837, 0.012, 1, 0, 0.8, 0)]

Rgap_result = np.zeros(len(Rgap_in))
for i in range(len(Rgap_in)):
    Rgap_result[i] = Gap_Resistance(Rgap_in[i])

print (Rgap_result)

### 5.1.3.3. 表面熱伝達抵抗

説明：グレージングの表面温度と周囲の放射温度から表面熱伝達抵抗を計算する。

### 入力値  
$ \theta_{sur} $ ：グレージングの表面温度 (℃)  
$ \theta_{r} $ ：周囲の放射温度 (℃)  
$ \epsilon_{sur} $ ：表面放射率   
cnv_type：対流熱伝達のflag（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）  

### 出力値
$ R_{sur} $ ：表面熱伝達抵抗 (m<sup>2</sup>･K/W)  

### 計算方法  

#### データ構造と値の範囲  

$ \theta_{sur} $・$ \theta_{r} $・$ \epsilon_{sur} $・cnv_typeの組み合わせを1つの変数（タプル）で表し"shc_input"とする。  

In [ ]:
shc_input = namedtuple('shc_input','shc_Tmcs shc_Tmcr shc_eps cnv_type')

# shc_Tmcs：グレージングの表面温度（℃）
# shc_Tmcr：周囲の放射温度（℃）
# shc_eps：表面放射率
# cnv_type：対流熱伝達のflag（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）

#### 表面熱伝達抵抗の計算  

（参考：JIS A 2103：2014 6. 境界条件）  

表面熱伝達抵抗の計算方法を以下に示す。 

$ \begin{align}R_{sur} = \frac{1}{h_{r,sur}+h_{c,sur}}\end{align}$ ･･････････････････････････････････ (5.1.3.3-a)  

$ \begin{align}h_{r,sur} = \frac{\epsilon_{sur}･\sigma･(T^4_{sur}-T^4_{r})}{T_{sur}-T_{r}}\end{align}$ ････････････････････････････ (5.1.3.3-b)  

ここで、  

$ R_{sur} $ ：表面熱伝達抵抗 (m<sup>2</sup>･K/W)  
$ h_{r,sur} $ ：表面放射熱伝達率 [W/(m<sup>2</sup>･K)]  
$ h_{c,sur} $ ：表面対流熱伝達率[W/(m<sup>2</sup>･K)]  
$ \epsilon_{sur} $ ：表面放射率  
$ \sigma $ ：ステファン・ボルツマン定数（＝5.67×10<sup>-7</sup>）[W/(m<sup>2</sup>･K<sup>4</sup>)]  
$ T_{sur} $ ：グレージングの表面温度（$T_{sur}=\theta_{sur}+273.15$） (K)   
$ T_{r} $ ：周囲の放射温度（$T_{r}=\theta_{r}+273.15$）(K)   

である。  
なお、本仕様書では、放射温度と空気温度は等しいものとして扱う。  
また、表面対流熱伝達率は、季節や室内外条件に応じた以下の値とする（参考：JIS A 2103：2014 表4）。

| 季節 | 室内外 | 表面対流熱伝達率 $h_{c,sur}$ [W/(m<sup>2</sup>･K)] |
| :---: | :---: | ---: |
| 夏期 | 室内 | 2.5 |
|  | 屋外 | 8.0 |
| 冬期 | 室内 | 3.6 |
|  | 屋外 | 20.0 |

In [ ]:
def Surface_Resistance(L):

    Tms = L.shc_Tmcs + 273.15
    Tmr = L.shc_Tmcr + 273.15
    sgm = 5.67 * 10.0 ** (-8.0)
    shc_hr = L.shc_eps * sgm * (Tms ** 4 - Tmr ** 4) / (Tms - Tmr)
    
    shc_hc = [2.5, 8.0, 3.6, 20.0]

    return 1.0 / (shc_hr + shc_hc[L.cnv_type])

### 計算例

In [ ]:
shc_in = [shc_input(16., 20., 0.837, 2),
          shc_input(4., 0., 0.837, 3)]

shc_result = np.zeros(len(shc_in))
for i in range(len(shc_in)):
    shc_result[i] = Surface_Resistance(shc_in[i])

print (shc_result)

### 5.1.3.4. 熱平衡

説明：グレージング複合体の熱平衡モデルによって面材間の表面温度と熱抵抗を計算する。

### 入力値  

$ d_{gl,j,l} $ ：層$j$の面材を構成する材料$l$の厚さ (m)  
$ \lambda_{gl,j,l} $ ：層$j$の面材を構成する材料$l$の熱伝導率 [W/(m･K)]  
$ I_{\alpha,j} $ ：層$j$の面材の吸収日射量 (W/m<sup>2</sup>)  
$ \epsilon_{gl,f} $ ：層$j$の面材の正面側の放射率  
$ \epsilon_{gl,b} $ ：層$j$の面材の背面側の放射率  
$ d_{gap,j} $ ：層$j-1$と層$j$の面材間の中空層の厚さ (m)  
gp_type1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gp_type2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）  
gp_ratio1：気体1の容積割合（0.0～1.0）  
gp_dir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）  
season：季節のflag（夏期：0、冬期：1）  
$ \theta_{ex} $ ：外気温 (℃)  
$ \theta_{in} $ ：室温 (℃)  

### 出力値
$ \theta_{f,j} $ ：層$j$の面材の正面側表面温度 (℃)  
$ \theta_{b,j} $ ：層$j$の面材の背面側表面温度 (℃)  
$ R_{s,j-1,j} $ ：層$j-1$と層$j$の面材間の中空層の熱抵抗 (m<sup>2</sup>･K/W)  
$ R_{sur,ex} $ ：屋外側表面熱伝達抵抗 (m<sup>2</sup>･K/W)  
$ R_{sur,in} $ ：室内側表面熱伝達抵抗 (m<sup>2</sup>･K/W)  

### 計算方法  

#### データ構造と値の範囲  

$ d_{gl} $・$ \lambda_{gl} $・$ I_{\alpha} $・$ \epsilon_{gl,f} $・$ \epsilon_{gl,b} $の組み合わせをグレージング層の物性値として1つの変数（タプル）で表し"glp_input"とする。  
$ d_{gap} $・gp_type_1・gp_type_2・gp_ratio_1・gp_dirの組み合わせをグレージング層間の中空層の物性値として1つの変数（タプル）で表し"gpp_input"とする。  
season・$ \theta_{ex} $・$ \theta_{in} $の組み合わせを気象条件として1つの変数（タプル）で表し"wp_input"とする。  


In [ ]:
glp_input = namedtuple('glp_input','gl_d gl_lmd gl_ia gl_epf gl_epb')
#gl_d：グレージング層jの厚さ(m)
#gl_lmd：グレージング層jの熱伝導率[W/(m・K)]
#gl_ia：グレージング層jの吸収日射量(W/m2)
#gl_epf：グレージング層jの正面側の放射率
#gl_epb：グレージング層jの背面側の放射率

gpp_input = namedtuple('gpp_input','gp_d gp_gas1 gp_gas2 gp_gas1r gp_dir')
#gp_d：グレージング層間の中空層の厚さ(m)
#gp_type1：気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
#gp_type2：気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
#gp_ratio1：気体1の容積割合（0.0～1.0）
#gp_dir：中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）

wp_input = namedtuple('wp_input','season te ti')
#season：季節のflag（夏期：0、冬期：1）
#te：外気温(℃)
#ti：室温（℃）

#### 表面温度と熱抵抗の収束計算  

（参考：JIS A 2103：2014 5.4.4 面材間の中空層の熱抵抗）  

各面材間における日射吸収と熱伝達との熱平衡の計算方法を以下に示す。

$ \begin{align}\frac{I_{\alpha,j}}{2} = \frac{\theta_{f,j} - \theta_{b,j-1}}{R_{s,j}(\theta_{b,j-1}, \theta_{f,j})} + \frac{\theta_{f,j} - \theta_{b,j}}{R_{gl,j}}\end{align}$ ･･････････････････････ (5.1.3.3-a)  

$ \begin{align}\frac{I_{\alpha,j}}{2} = \frac{\theta_{b,j} - \theta_{f,j}}{R_{gl,j}} + \frac{\theta_{b,j} - \theta_{f,j+1}}{R_{s,j+1}(\theta_{b,j}, \theta_{f,j+1})} \end{align}$ ････････････････････ (5.1.3.3-b)  

ここで、  

$ I_{\alpha,j} $ ：層$j$での吸収日射量 (W/m<sup>2</sup>)  
$ \theta_{f,j} $ ：層$j$の面材の正面側表面温度 (℃)  
$ \theta_{b,j} $ ：層$j$の面材の背面側表面温度 (℃)  
$ R_{gl,j} $ ：層$j$の面材の熱抵抗 (m<sup>2</sup>･K/W)  
$ R_{s,j}(\theta_{b,j-1}, \theta_{f,j}) $ ：層$j-1$と層$j$の面材間の中空層の熱抵抗 (m<sup>2</sup>･K/W) 

である。 

$ \theta_{b,-1} = \theta_{ex} $、$ \theta_{f,n} = \theta_{in} $、$ R_{s,0} = R_{sur,ex} $、$ R_{s,n} = R_{sur,in} $として、式(5.1.3.3-a)と式(5.1.3.3-b)から得られる$n$層のグレージング複合体の熱平衡の連立方程式を以下に示す。  

$ 
\begin{pmatrix} 
\frac{1}{R_{sur,ex}}+\frac{1}{R_{gl,0}} & -\frac{1}{R_{gl,0}} &&&&& \\
-\frac{1}{R_{gl,0}} & \frac{1}{R_{gl,0}}+\frac{1}{R_{s,1}} & -\frac{1}{R_{s,1}} &&&& \\
&-\frac{1}{R_{s,1}} & \frac{1}{R_{s,1}}+\frac{1}{R_{gl,1}} & -\frac{1}{R_{gl,1}} &&O& \\
&&-\frac{1}{R_{gl,1}} & \frac{1}{R_{gl,1}}+\frac{1}{R_{s,2}} &&& \\
&&&&&\ddots&& \\
&&&&\ddots&&& \\
&O&&\ddots&&&& \\
&&&&& \frac{1}{R_{s,n-1}}+\frac{1}{R_{gl,n-1}} & -\frac{1}{R_{gl,n-1}}  \\
&&&&&-\frac{1}{R_{gl,n-1}} & \frac{1}{R_{gl,n-1}}+\frac{1}{R_{sur,in}}  \\
\end{pmatrix} 
\begin{pmatrix} 
\theta_{f,0} \\
\theta_{b,0} \\
\theta_{f,1} \\
\vdots \\
\vdots \\
\vdots \\
\theta_{b,n-1} \\
\end{pmatrix} 
= 
\begin{pmatrix} 
\frac{I_{\alpha,0}}{2} + \frac{\theta_{ex}}{R_{sur,ex}} \\ 
\frac{I_{\alpha,0}}{2} \\
\frac{I_{\alpha,1}}{2} \\ 
\frac{I_{\alpha,1}}{2} \\
\vdots \\
\vdots \\
\vdots \\
\frac{I_{\alpha,n-1}}{2} \\ 
\frac{I_{\alpha,n-1}}{2} + \frac{\theta_{in}}{R_{sur,in}} \\
\end{pmatrix} 
$ ････ (5.1.3.3-c)  

![image1.png][attached1]
[attached1]:img\image1.png  

<div style="text-align: center;">
$n$層のグレージング複合体の熱平衡モデル
</div>


ここで、$ R_{gl,j} $ = $ R_{2j+1} $、$ R_{s,j}(\theta_{b,j-1}, \theta_{f,j}) $ = $ R_{2j} $、$ \theta_{f,j} $ = $ \theta_{2j+1} $、$ \theta_{b,j} $ = $ \theta_{2(j+1)} $、$ \frac{I_{\alpha,j}}{2} = q_{\alpha,2j+1} $、$ \frac{I_{\alpha,j}}{2} = q_{\alpha,2(j+1)} $と置き換えた場合の面材と中空層の層の番号の関係を以下に示す。

$ 
\begin{pmatrix} 
\frac{1}{R_{0}}+\frac{1}{R_{1}} & -\frac{1}{R_{1}} &&&&& \\
-\frac{1}{R_{1}} & \frac{1}{R_{1}}+\frac{1}{R_{2}} & -\frac{1}{R_{2}} &&&& \\
&-\frac{1}{R_{2}} & \frac{1}{R_{2}}+\frac{1}{R_{3}} & -\frac{1}{R_{3}} &&O& \\
&&-\frac{1}{R_{3}} & \frac{1}{R_{3}}+\frac{1}{R_{4}} &&& \\
&&&&&\ddots&& \\
&&&&\ddots&&& \\
&O&&\ddots&&&& \\
&&&&& \frac{1}{R_{2(n-1)}}+\frac{1}{R_{2(n-1)+1}} & -\frac{1}{R_{2(n-1)+1}}  \\
&&&&&-\frac{1}{R_{2(n-1)+1}} & \frac{1}{R_{2(n-1)+1}}+\frac{1}{R_{2n}}  \\
\end{pmatrix} 
\begin{pmatrix} 
\theta_{1} \\
\theta_{2} \\
\theta_{3} \\
\vdots \\
\vdots \\
\vdots \\
\theta_{2n} \\
\end{pmatrix} 
= 
\begin{pmatrix} 
\frac{q_{\alpha,1}}{2} + \frac{\theta_{0}}{R_{0}} \\ 
\frac{q_{\alpha,2}}{2} \\
\frac{q_{\alpha,3}}{2} \\ 
\frac{q_{\alpha,4}}{2} \\
\vdots \\
\vdots \\
\vdots \\
\frac{q_{\alpha,2(n-1)+1}}{2} \\ 
\frac{q_{\alpha,2n}}{2} + \frac{\theta_{2n+1}}{R_{2n}} \\
\end{pmatrix} 
$ ････ (5.1.3.3-d)  

![image2.png][attached2]
[attached2]:img\image2.png

<div style="text-align: center;">
$n$層のグレージング複合体の熱平衡モデル（記号置換後）
</div>

式(5.1.3.3-d)の左辺の熱抵抗$R$に関する行列は正定値対称行列であるため、修正コレスキー分解によって熱抵抗$R$及び温度$\theta$を求める。

In [ ]:
def Heat_balance(L, M, N):
    
    #面材の層数
    n = len(L.gl_d)
    
    #面材の熱抵抗
    Rgl = np.zeros(n)
    for i in range(n):
        Rgl_in = gl_input(L.gl_d[i], L.gl_lmd[i])
        Rgl[i] = Glazing_Resistance(Rgl_in)
    
    q_a = np.zeros(2 * n + 1)
    q_a[0] = 0.
    for i in range(1, n + 1):   
        q_a[2 * (i - 1) + 1] = L.gl_ia[i - 1] / 2.
        q_a[2 * i] = L.gl_ia[i - 1] / 2.
        
    r_t = np.zeros(2 * n + 2)
    told = np.zeros(2 * n + 2)
    r_R = np.zeros(2 * n + 1)
    Rold = np.zeros(2 * n + 1)

    #温度の初期値
    for i in range(0, 2 * n + 2):
        r_t[i] = N.te +(N.ti - N.te) * i / (2 * n + 1)
        told[i] = r_t[i]
    #print(t)

    while True:
        #前回の温度
        for i in range(2 * n + 2):
            told[i] = r_t[i]
    
        #前回の熱抵抗
        for i in range(2 * n + 1):
            Rold[i] = r_R[i]

        #面材間の中空層の熱抵抗
        Rs = np.zeros(n - 1)
        for i in range(n - 1):
            Rs_in = gap_input(r_t[2 * (i + 1)], r_t[2 * (i + 1) + 1], L.gl_epb[i], L.gl_epf[i + 1], \
                              M.gp_d[i], M.gp_gas1[i], M.gp_gas2[i], M.gp_gas1r[i], M.gp_dir[i])
            Rs[i] = Gap_Resistance(Rs_in)

        #表面熱伝達抵抗（0：夏期室内、1：夏期屋外、2：冬期室内、3：冬期屋外）
        Rsur_in = [shc_input(r_t[1], N.te, L.gl_epf[0], 2 * N.season + 1), 
                   shc_input(r_t[2 * n], N.ti, L.gl_epb[n - 1], 2 * N.season)]

        Rsur = np.zeros(len(Rsur_in))
        for i in range(len(Rsur_in)):
            Rsur[i] = Surface_Resistance(Rsur_in[i])

        #熱抵抗の配列
        r_R[0] = Rsur[0]
        for i in range(n):
            r_R[2 * i + 1] = Rgl[i]
        for i in range(n - 1):
            r_R[2 * (i + 1)] = Rs[i]
        r_R[2 * n] = Rsur[1]

        #行列式の生成
        ca = np.zeros((2 * n, 2 * n))
        cb = np.zeros((2 * n))
        cx = np.zeros((2 * n))

        for i in range(2 * n + 1):
            if r_R[i] == 0.:
                r_R[i] = 10. ** (-10.)
        for i in range(2 * n):
            ca[i][i] = 1/ r_R[i] + 1/ r_R[i + 1]
            if i == 0:
                cb[i] = q_a[i + 1] + r_t[i] / r_R[i]
            elif i == 2 * n - 1:
                cb[i] = q_a[i + 1] + r_t[i + 2] / r_R[i + 1]
            else:
                cb[i] = q_a[i + 1]

        for i in range(2 * n - 1):
            ca[i + 1][i] = - 1/ r_R[i + 1]
            ca[i][i + 1] = - 1/ r_R[i + 1]
        #修正コレスキー分解
        cx = modified_Cholesky_decomposition(ca, cb)

        #表面温度の取得
        for i in range(0, 2 * n):
            r_t[i + 1] = cx[i]

        #温度の収束判定
        tcdt = 1
        cdt =  np.zeros(2 * n + 2)
        for i in range(2 * n + 2):
            if abs(r_t[i] - told[i]) < 10 **(-10):
                cdt[i] = 1
            else:
                cdt[i] = 0
            tcdt = tcdt * cdt[i]

        #熱抵抗の収束判定
        tcdR = 1
        cdR =  np.zeros(2 * n + 1)
        for i in range(0, 2 * n + 1):
            if abs(r_R[i] - Rold[i]) < 10 **(-10):
                cdR[i] = 1
            else:
                cdR[i] = 0
            tcdR = tcdR * cdR[i]

        tcd = tcdt + tcdR
        if tcd == 2:
            break

    return r_t, r_R

#### 修正コレスキー分解

修正コレスキー分解の解法を以下に示す。  

$ \begin{align} \boldsymbol{Ax} = \boldsymbol{b} \end{align}$ ･･･････････････････････････････････････････････ (5.1.3.3-e)  

ここで、  

$ 
\boldsymbol{A}=
\begin{pmatrix} 
a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
\vdots  & \vdots  & \ddots & \vdots  \\
a_{n,1} & a_{n,2} & \cdots & a_{n,n} \\
\end{pmatrix}
$
、
$ 
\boldsymbol{x}=
\begin{pmatrix} 
x_{1} \\
x_{2} \\
\vdots \\
x_{n} \\
\end{pmatrix}
$
、
$ 
\boldsymbol{b}=
\begin{pmatrix} 
b_{1} \\
b_{2} \\
\vdots \\
b_{n} \\
\end{pmatrix}
$

ただし、$ \begin{align}a_{i,j} = a_{j,i}\end{align}$  

行列$\boldsymbol{A}$を以下のように分解する。

$ \begin{align} \boldsymbol{A} = \boldsymbol{LDL}^T\end{align}$ ････････････････････････････････････････････ (5.1.3.3-f)  

ここで、  

$ 
\boldsymbol{L}=
\begin{pmatrix} 
1 &&&& \\
l_{2,1} & 1 &&O& \\
l_{3,1} & l_{3,2} & 1 && \\
\vdots  & \vdots  & \vdots  & \ddots & \\
l_{n,1} & l_{n,2} & l_{n,3} & \cdots & 1 \\
\end{pmatrix}
$
、
$ 
\boldsymbol{D}=
\begin{pmatrix} 
d_{1,1} &&&& \\
& d_{2,2} &&O& \\
&& \ddots && \\
&O&& \ddots & \\
&&&& d_{n,n} \\
\end{pmatrix}
$
、
$ 
\boldsymbol{L}^T=
\begin{pmatrix} 
1 & l_{1,2} & l_{1,3} & \cdots & l_{1,n} \\
& 1 & l_{2,2} & \cdots & l_{2,n} \\
&& 1 & \cdots & l_{3,n} \\
&O&& \ddots &\vdots \\
&&&& 1 \\
\end{pmatrix}
$  

ただし、$ \begin{align}l_{i,j} = l_{j,i}\end{align}$  

この関係を整理すると、以下のようになる。  

$ \begin{align} d_{1,1} = a_{1,1} \end{align}$ ････････････････････････････････････････････ (5.1.3.3-g)  

$ \begin{align}l_{k,i} = \frac{a_{k,i} - \sum_{j=1}^{i-1} l_{k,j} ･ l_{i,j} ･ d_{j,j}}{d_{i,i}}\end{align}$ ･･･････････････････････････ (5.1.3.3-h)  

$ \begin{align}d_{k,k} = a_{k,k} - \sum_{i=1}^{k-1} l_{k,i}^2 ･ d_{i,i}\end{align}$ ････････････････････････････････ (5.1.3.3-i)  

式(5.1.3.3-f)より、  

$ \begin{align} \boldsymbol{LDL}^T \boldsymbol{x} = \boldsymbol{b} \end{align}$ ･･･････････････････････････････････････････ (5.1.3.3-j)  

$ \begin{align} \boldsymbol{DL}^T \boldsymbol{x} = \boldsymbol{y} \end{align}$ とすると、

$ \begin{align} \boldsymbol{Ly} = \boldsymbol{b} \end{align}$ ･･･････････････････････････････････････････････ (5.1.3.3-k)  

ここで、  

$ 
\boldsymbol{y}=
\begin{pmatrix} 
y_{1} \\
y_{2} \\
\vdots \\
y_{n} \\
\end{pmatrix}
$

この関係を整理すると、以下のようになる。  

$ \begin{align}y_i = b_i\end{align}$ ($i=1$)･････････････････････････････････････････ (5.1.3.3-l)  

$ \begin{align}y_i = b_i - \sum_{j=1}^{n-1} l_{i,j} ･ y_j\end{align}$ ($2\leq i \leq n$)･････････････････････････ (5.1.3.3-m)  

$ \begin{align}x_i = \frac{y_i}{d_{i,i}}\end{align}$ ($i=n$)･･･････････････････････････････････････ (5.1.3.3-n)  

$ \begin{align}x_i = \frac{y_i - d_{i,i} ･ \sum_{j=i+1}^{n} l_{j,i} ･ x_j}{d_{i,i}}\end{align}$ ($1\leq i \leq n-1$)･････････････ (5.1.3.3-o)

In [ ]:
#修正コレスキー分解
def modified_Cholesky_decomposition(a, b):

    n = len(a)
    d = np.zeros(n)
    l = np.zeros((n,n))
    x = np.zeros(n)
    y = np.zeros(n)

    d[0] = a[0][0]

    for i in range(n):
        l[i][i] = 1.

    for k in range(1,n):
        s = 0.
        t = 0.
        for i in range(k):
            for j in range(i):
                s -= l[k][j] * l[i][j] * d[j]
            l[k][i] = (a[k][i] + s) / d[i]

        for i in range(k):
            t -= l[k][i] ** 2. * d[i]

        d[k] = a[k][k] + t

    y[0] = b[0]
    for i in range(1,n):
        u = 0.
        for j in range(n-1):
            u -= l[i][j] * y[j]

        y[i] = b[i] + u

    x[n-1] = y[n-1] / d[n-1]
    for i in range(n-2,-1,-1):
        v = 0.
        for j in range(n-1,i,-1):
            v -= l[j][i] * x[j]

        x[i] = y[i] / d[i] + v

    return x

### 計算例

In [ ]:
#グレージングの厚さ
gl_d = [[0.003, 0.006], [0.003], [0.003]]

#グレージングの熱伝導率
gl_lmd = [[1., 0.5], [1.], [1.]]

#各層の吸収日射量
gl_ia = [9.55935027, 6.8267886, 4.76774099]

#各層の正面側の放射率
gl_epf = [0.837, 0.837, 0.837]

#各層の背面側の放射率
gl_epb = [0.837, 0.837, 0.837]

#中空層の厚さ
gp_d = [0.012, 0.012]

#中空層の気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
gp_gas1 = [0, 0]

#中空層の気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
gp_gas2 = [0, 0]

# 中空層の気体1の容積割合（0.0～1.0）
gp_gas1r = [1., 1.]

# 中空層の熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、2：中空層が45度で熱流方向が上向き）
gp_dir = [0, 0]

season = 0 #夏期：0、冬期：1
if season == 0:
    te = 30.  #外気温
    ti = 25.  #室温
elif season == 1:
    te = 0.   #外気温
    ti = 20.  #室温

glp_in = glp_input(gl_d, gl_lmd, gl_ia, gl_epf, gl_epb)
gpp_in = gpp_input(gp_d, gp_gas1, gp_gas2, gp_gas1r, gp_dir)
wp_in = wp_input(season, te, ti)

hbr = Heat_balance(glp_in, gpp_in, wp_in)

tglz = hbr[0][0 : 2 * (len(gl_d) + 1)]
Rglz = hbr[1][0 : 2 * len(gl_d) + 1]

print ('tglz', tglz)
print ('Rglz', Rglz)

### 5.1.4. 全体計算

### 5.1.4.1. 日射量の単位変換  

### 5.1.4.1.1. MJ/(m<sup>2</sup>･h) → W/m<sup>2</sup> 

説明：日射量の単位を MJ/(m<sup>2</sup>･h) から W/m<sup>2</sup> に変換する。

### 入力値  

$ I_{Joule} $ ：日射量 [MJ/(m<sup>2</sup>･h)]  

### 出力値  

$ I_{Watt} $ ：日射量 (W/m<sup>2</sup>)  

### 計算方法  

#### 日射量の単位変換

日射量の入力値の単位がMJ/(m<sup>2</sup>･h)の場合には、W/m<sup>2</sup>に単位変換したのち全体計算を行うものとする。  
単位の変換方法を以下に示す。

$ \begin{align}I_{Watt} = \frac{10 ^ 6}{3600} ･ I_{Joule}\end{align}$ ･･････････････････････････････････････････････････････････････････････ (5.1.4.1.1-a)  

In [ ]:
def transform_unit_joule2watt(I):
    
    return I * 10. ** 6. / 3600.

### 計算例

In [ ]:
I_joule = 4.9212
I_watt = transform_unit_joule2watt(I_joule)

print (I_watt)

### 5.1.4.1.2. W/m<sup>2</sup> → MJ/(m<sup>2</sup>･h) 

説明：日射量の単位を W/m<sup>2</sup> から MJ/(m<sup>2</sup>･h) に変換する。

### 入力値  

$ I_{Watt} $ ：日射量 (W/m<sup>2</sup>)  

### 出力値  

$ I_{Joule} $ ：日射量 [MJ/(m<sup>2</sup>･h)]  

### 計算方法  

#### 日射量の単位変換

全体計算を行ったのち日射量の単位はW/m<sup>2</sup>からMJ/(m<sup>2</sup>･h)に単位変換する。  
単位の変換方法を以下に示す。

$ \begin{align}I_{Joule} = \frac{3600}{10 ^ 6} ･ I_{Watt}\end{align}$ ･･････････････････････････････････････････････････････････････････････ (5.1.4.1.2-a)  

In [ ]:
def transform_unit_watt2joule(I):
    
    return I * 3600. / 10. ** 6.

### 計算例

In [ ]:
I_watt = 1367.
I_joule = transform_unit_watt2joule(I_watt)

print (I_joule)

### 5.1.4.2. 開口面入射日射量

説明：斜面直達日射量、斜面天空日射量、斜面地表面反射日射量から開口面入射日射量を計算する。

### 入力値  

$ I_{T,b} $ ：斜面（開口面）直達日射量 (W/m<sup>2</sup>)  
$ I_{T,d} $ ：斜面（開口面）天空日射量 (W/m<sup>2</sup>)  
$ I_{T,r} $ ：斜面（開口面）地表面反射日射量 (W/m<sup>2</sup>)  

### 出力値  

$ I_{T} $ ：開口面入射日射量 (W/m<sup>2</sup>)  

### 計算方法  

#### データ構造と値の範囲  

$ I_{T,b} $・$ I_{T,d} $・$ I_{T,r} $ の組み合わせを入力条件として1つの変数（タプル）で表し"ita_input"とする。  

In [ ]:
ita_input = namedtuple('ita_input','ita_Itb ita_Itd ita_Itr')
#ita_Itb：斜面（開口面）直達日射量 (W/m2)
#ita_Itd：斜面（開口面）天空日射量 (W/m2)
#ita_Itr：斜面（開口面）地表面反射日射量 (W/m2)

#### 開口面入射日射量の計算

開口面入射日射量の計算方法を以下に示す。

$ \begin{align}I_{T} = I_{T,b} + I_{T,d} + I_{T,r} \end{align}$ ･･････････････････････････････････････････････････････････････････････････････ (5.1.4.2-a)  

In [ ]:
def incident_sol(L):
    
    return L.ita_Itb + L.ita_Itd + L.ita_Itr

### 計算例

In [ ]:
ita_in = ita_input(2.6840261248 * 10. ** 6. / 3600., 0.1181932153 * 10. ** 6. / 3600., 0.094 * 10. ** 6. / 3600.)

I_in = incident_sol(ita_in)                           #斜面（開口面）全天日射量

print (I_in)

### 5.1.4.3. 開口面透過日射量

説明：開口面日射量（直達・天空・地表面反射）とグレージング複合体の日射透過率から開口面透過日射量を計算する。

### 入力値  

$ I_{T,b} $ ：斜面（開口面）直達日射量 (W/m<sup>2</sup>)  
$ I_{T,d} $ ：斜面（開口面）天空日射量 (W/m<sup>2</sup>)  
$ I_{T,r} $ ：斜面（開口面）地表面反射日射量 (W/m<sup>2</sup>)  
$ \tau_{glz,b} $ ：直達日射に対する日射透過率  
$ \tau_{glz,d,l,m} $ ：天空上の微小要素からの入射日射に対する日射透過率  
$ \tau_{glz,r,l,m} $ ：地表面上の微小要素からの入射日射に対する日射透過率  
$ F_{d,l,m} $ ：天空日射量に対する天空上の微小要素からの天空日射量の割合  
$ F_{r,l,m} $ ：地表面反射日射量に対する地表面上の微小要素からの地表面反射日射量の割合  

### 出力値  

$ I_{\tau} $ ：開口面透過日射量 (W/m<sup>2</sup>)  

### 計算方法  

#### データ構造と値の範囲  

$ \tau_{glz,b} $・$ \tau_{glz,d} $・$ \tau_{glz,r} $ の組み合わせを入力条件として1つの変数（タプル）で表し"tta_input"とする。  

In [ ]:
tta_input = namedtuple('tta_input','tta_taub tta_taud tta_taur')
#tta_taub：直達成分の日射透過率
#tta_taud：天空成分の日射透過率
#tta_taur：地表面反射成分の日射透過率

#### 透過日射量の計算  

透過日射量の計算方法を以下に示す。

$ \begin{align}I_{\tau} = I_{T,b}･\tau_{glz,b} + I_{T,d}･\tau_{glz,d} + I_{T,r}･\tau_{glz,r} \end{align}$ ････････････････････ (5.1.4.3-a)  

$ \begin{align}\tau_{glz,d} = \sum_{l=0}^{N_{d,lat}-1}\sum_{m=0}^{N_{d,lon}-1} F_{d,l,m}･\tau_{glz,d,l,m} \end{align}$ ････････････････････ (5.1.4.3-b)  

$ \begin{align}\tau_{glz,r} = \sum_{l=0}^{N_{r,lat}-1}\sum_{m=0}^{N_{r,lon}-1} F_{r,l,m}･\tau_{glz,r,l,m} \end{align}$ ････････････････････ (5.1.4.3-c)  

In [ ]:
#開口面透過日射量
def transmit_sol(L, M):
    
    return L.ita_Itb * M.tta_taub + L.ita_Itd * M.tta_taud + L.ita_Itr * M.tta_taur

#直達日射での透過率
def actual_trans_beam(L, phi_b):
    
    r_taub = 0.
    ly_in_b = np.empty(len(L), dtype = tuple)
    for j in range(0, len(L)):
        ly_in_b[j] = layer_input(glass_TRSf(L[j], phi_b), glass_TRSb(L[j], phi_b), glass_REFf(L[j], phi_b), glass_REFb(L[j], phi_b))
    
    r_taub = Sol_Trs_All(ly_in_b)
                
    return r_taub

#天空日射での透過率
def actual_trans_diffuse(L, Nd_lat, Nd_lon):

    F_d = np.empty((Nd_lat, Nd_lon))
    phi_d = np.empty((Nd_lat, Nd_lon))
    tau_d = np.empty((Nd_lat, Nd_lon))
    r_taud = 0.
    ly_in_d = np.empty(len(L), dtype = tuple)
    for l in range(0, Nd_lat):
        for m in range(0, Nd_lon):
            dd_in = dd_input(Nd_lat, Nd_lon, l, m)
            F_d[l][m] = DivisionDiffuseRatio.division_diffuse_ratio(dd_in)[0]   #割合
            phi_d[l][m] = DivisionDiffuseRatio.division_diffuse_ratio(dd_in)[3] #入射角
            
            for j in range(0, len(L)):
                ly_in_d[j] = layer_input(glass_TRSf(L[j], phi_d[l][m]), glass_TRSb(L[j], phi_d[l][m]), \
                                         glass_REFf(L[j], phi_d[l][m]), glass_REFb(L[j], phi_d[l][m]))

            tau_d[l][m] = Sol_Trs_All(ly_in_d)

            r_taud += F_d[l][m] * tau_d[l][m]
               
    return r_taud

#地表面反射日射での透過率
def actual_trans_reflect(L, Nr_lat, Nr_lon):

    F_r = np.empty((Nr_lat, Nr_lon))
    phi_r = np.empty((Nr_lat, Nr_lon))
    tau_r = np.empty((Nr_lat, Nr_lon))
    r_taur =0.
    ly_in_r = np.empty(len(L), dtype = tuple)  
    for l in range(0, Nr_lat):
        for m in range(0, Nr_lon):
            dr_in = dr_input(Nr_lat, Nr_lon, l, m)
            F_r[l][m] = DivisionDiffuseRatio.division_reflect_ratio(dr_in)[0]   #割合
            phi_r[l][m] = DivisionDiffuseRatio.division_reflect_ratio(dr_in)[3] #入射角

            for j in range(0, len(L)):
                ly_in_r[j] = layer_input(glass_TRSf(L[j], phi_r[l][m]), glass_TRSb(L[j], phi_r[l][m]), \
                                         glass_REFf(L[j], phi_r[l][m]), glass_REFb(L[j], phi_r[l][m]))

            tau_r[l][m] = Sol_Trs_All(ly_in_r)

            r_taur += F_r[l][m] * tau_r[l][m]
                
    return r_taur

### 計算例

In [ ]:
#ガラス光学特性
gl_in = [glass_input(0.815, 0.815, 0.072, 0.072, 0, 0, 0),
         glass_input(0.815, 0.815, 0.072, 0.072, 0, 0, 0),
         glass_input(0.815, 0.815, 0.072, 0.072, 0, 0, 0)]

#斜面日射量
ita_in = ita_input(2.6840261248 * 10. ** 6. / 3600., 0.1181932153 * 10. ** 6. / 3600., 0.094 * 10. ** 6. / 3600.)

#直達日射の入射角
phi_b = 31.6403837639

#天空分割数
Nd_lat = 90
Nd_lon = 180

#地表面反射分割数
Nr_lat = 90
Nr_lon = 180

tau_b = actual_trans_beam(gl_in, phi_b)
tau_d = actual_trans_diffuse(gl_in, Nd_lat, Nd_lon)
tau_r = actual_trans_reflect(gl_in, Nr_lat, Nr_lon)

tta_in = tta_input(tau_b, tau_d, tau_r)
I_tau = transmit_sol(ita_in, tta_in)                  #斜面（開口面）透過日射量
    
print (I_tau)

### 5.1.4.4. 各層の吸収日射量

説明：開口面日射量（直達・天空・地表面反射）とグレージング複合体各層の日射吸収率から各層の吸収日射量を計算する。

### 入力値  

$ I_{T,b} $ ：斜面（開口面）直達日射量 (W/m<sup>2</sup>)  
$ I_{T,d} $ ：斜面（開口面）天空日射量 (W/m<sup>2</sup>)  
$ I_{T,r} $ ：斜面（開口面）地表面反射日射量 (W/m<sup>2</sup>)  
$ \alpha_{glz,b,j} $ ：直達日射に対する層$j$の日射吸収率  
$ \alpha_{glz,d,j,l,m} $ ：天空上の微小要素からの入射日射に対する層$j$の日射吸収率  
$ \alpha_{glz,r,j,l,m} $ ：地表面上の微小要素からの入射日射に対する層$j$の日射吸収率  
$ F_{d,l,m} $ ：天空日射量に対する天空上の微小要素からの天空日射量の割合  
$ F_{r,l,m} $ ：地表面反射日射量に対する地表面上の微小要素からの地表面反射日射量の割合  

### 出力値  

$ I_{\alpha,j} $ ：層$j$での吸収日射量 (W/m<sup>2</sup>)  

### 計算方法  

#### データ構造と値の範囲  

$ \alpha_{glz,b,j} $・$ \alpha_{glz,d,j} $・$ \alpha_{glz,r,j} $ の組み合わせを入力条件として1つの変数（タプル）で表し"ata_input"とする。  

In [ ]:
ata_input = namedtuple('ata_input','ata_absb ata_absd ata_absr')
#ata_absb：直達成分の日射吸収率
#ata_absd：天空成分の日射吸収率
#ata_absr：地表面反射成分の日射吸収率

#### 層$j$での吸収日射量の計算  

層$j$での吸収日射量の計算方法を以下に示す。

$ \begin{align}I_{\alpha,j} = I_{T,b}･\alpha_{glz,b,j} + I_{T,d}･\alpha_{glz,d} + I_{T,r}･\alpha_{glz,r} \end{align}$ ･･････････････････････ (5.1.4.4-a)  

$ \begin{align}\alpha_{glz,d} = \sum_{l=0}^{N_{d,lat}-1}\sum_{m=0}^{N_{d,lon}-1} F_{d,l,m}･\alpha_{glz,d,j,l,m} \end{align}$ ････････････････････ (5.1.4.4-b)  

$ \begin{align}\alpha_{glz,r} = \sum_{l=0}^{N_{r,lat}-1}\sum_{m=0}^{N_{r,lon}-1} F_{r,l,m}･\alpha_{glz,r,j,l,m} \end{align}$ ････････････････････ (5.1.4.4-c)  

In [ ]:
#層jの吸収日射量
def absorb_sol(L, M):
    
    return L.ita_Itb * M.ata_absb + L.ita_Itd * M.ata_absd + L.ita_Itr * M.ata_absr

#直達日射での層jの吸収率
def actual_abs_beam(L, phi_b):
    
    r_absb = np.zeros(len(L))
    ly_in_b = np.empty(len(L), dtype = tuple)
    for j in range(0, len(L)):
        ly_in_b[j] = layer_input(glass_TRSf(L[j], phi_b), glass_TRSb(L[j], phi_b), glass_REFf(L[j], phi_b), glass_REFb(L[j], phi_b))
    
    for j in range(0, len(L)):
        r_absb[j]  = Abs_Multi_Lay(ly_in_b, j)
                
    return r_absb

#天空日射での層jの吸収率
def actual_abs_diffuse(L, Nd_lat, Nd_lon):

    F_d = np.empty((Nd_lat, Nd_lon))
    phi_d = np.empty((Nd_lat, Nd_lon))
    abs_d = np.empty((len(L), Nd_lat, Nd_lon))
    r_absd = np.zeros(len(L))
    ly_in_d = np.empty(len(L), dtype = tuple) 
    for l in range(0, Nd_lat):
        for m in range(0, Nd_lon):
            dd_in = dd_input(Nd_lat, Nd_lon, l, m)
            F_d[l][m] = DivisionDiffuseRatio.division_diffuse_ratio(dd_in)[0]   #割合
            phi_d[l][m] = DivisionDiffuseRatio.division_diffuse_ratio(dd_in)[3] #入射角
            
            for j in range(0, len(L)):
                ly_in_d[j] = layer_input(glass_TRSf(L[j], phi_d[l][m]), glass_TRSb(L[j], phi_d[l][m]), \
                                         glass_REFf(L[j], phi_d[l][m]), glass_REFb(L[j], phi_d[l][m]))

            for j in range(0, len(L)):
                abs_d[j][l][m] = Abs_Multi_Lay(ly_in_d, j)
                r_absd[j] += F_d[l][m] * abs_d[j][l][m]
                
    return r_absd

#地表面反射日射での層jの吸収率
def actual_abs_reflect(L, Nd_lat, Nd_lon):

    F_r = np.empty((Nr_lat, Nr_lon))
    phi_r = np.empty((Nr_lat, Nr_lon))
    abs_r = np.empty((len(L), Nr_lat, Nr_lon))
    r_absr = np.zeros(len(L))
    ly_in_r = np.empty(len(L), dtype = tuple)   
    for l in range(0, Nr_lat):
        for m in range(0, Nr_lon):
            dr_in = dr_input(Nr_lat, Nr_lon, l, m)
            F_r[l][m] = DivisionDiffuseRatio.division_reflect_ratio(dr_in)[0]   #割合
            phi_r[l][m] = DivisionDiffuseRatio.division_reflect_ratio(dr_in)[3] #入射角

            for j in range(0, len(L)):
                ly_in_r[j] = layer_input(glass_TRSf(L[j], phi_r[l][m]), glass_TRSb(L[j], phi_r[l][m]), \
                                         glass_REFf(L[j], phi_r[l][m]), glass_REFb(L[j], phi_r[l][m]))

            for j in range(0, len(L)):
                abs_r[j][l][m] = Abs_Multi_Lay(ly_in_r, j)
                r_absr[j] += F_r[l][m] * abs_r[j][l][m]
                
    return r_absr

### 計算例

In [ ]:
#ガラス光学特性
gl_in = [glass_input(0.815, 0.815, 0.072, 0.072, 0, 0, 0),
         glass_input(0.815, 0.815, 0.072, 0.072, 0, 0, 0),
         glass_input(0.815, 0.815, 0.072, 0.072, 0, 0, 0)]

#斜面日射量
ita_in = ita_input(2.6840261248 * 10. ** 6. / 3600., 0.1181932153 * 10. ** 6. / 3600., 0.094 * 10. ** 6. / 3600.)

#直達日射の入射角
phi_b = 31.6403837639

#天空分割数
Nd_lat = 90
Nd_lon = 180

#地表面反射分割数
Nr_lat = 90
Nr_lon = 180

abs_b = np.zeros(len(gl_in))
abs_d = np.zeros(len(gl_in))
abs_r = np.zeros(len(gl_in))
I_abs = np.zeros(len(gl_in))
for j in range(0, len(gl_in)):
    abs_b[j] = actual_abs_beam(gl_in, phi_b)[j]
    abs_d[j] = actual_abs_diffuse(gl_in, Nd_lat, Nd_lon)[j]
    abs_r[j] = actual_abs_reflect(gl_in, Nr_lat, Nr_lon)[j]
    
    ata_in = ata_input(abs_b[j], abs_d[j], abs_r[j])
    I_abs[j] = absorb_sol(ita_in, ata_in)             #斜面（開口面）吸収日射量

print (I_abs)

### 5.1.4.5. 室内側へ流入する吸収日射量

説明：$ N_{lay} $層の面材からなるグレージング複合体の熱抵抗と層$j$の吸収日射量から室内側へ流入する吸収日射量を計算する。

### 入力値  

$ I_{\alpha,j} $ ：層$j$での吸収日射量（$0$～$N_{lay}-1$） (W/m<sup>2</sup>)  
$ R_{glz,k} $ ：グレージング複合体の$k$番目の熱抵抗（$0$番目を屋外側表面熱伝達抵抗、$2･j$番目をグレージング層$j-1$番目と$j$番目の間の中間層の熱抵抗、グレージング層の熱抵抗、$2･j+1$番目をグレージング層$j$番目の熱抵抗、$2･N_{lay}$番目を室内側表面熱伝達抵抗とした直列の熱抵抗）(m<sup>2</sup>･K/W)   

### 出力値  

$ I_{qi} $ ：室内側へ流入する吸収日射量 (W/m<sup>2</sup>)  

### 計算方法  

#### データ構造と値の範囲  
$ I_{\alpha} $・$ R_{glz} $の組み合わせを入力条件として1つの変数（タプル）で表し"qin_input"とする。  


In [ ]:
qin_input = namedtuple('qin_input','qin_ia qin_rglz')
#qin_ia：グレージング各層の吸収日射量(W/m2)
#qin_rglz：直列に並べたグレージング複合体の熱抵抗[(m・K2)/W]

#### 各層の吸収日射熱が室内側へ再放出される割合

（参考：JIS A 2103：2014 5.4.1 計算基礎式）

層$j$の面材の吸収日射熱が室内側へ再放出される割合$N_{in,j}$の計算方法を以下に示す。  

$ \begin{align} N_{in,j} = \frac {\sum_{k=0}^{2･j} R_{glz,k} + \frac {R_{glz,2･j+1}}{2}}{\sum_{k=0}^{2･N_{lay}} R_{glz,k}} \end{align}$ ･･･････････････････････････････････････････････････････････････････････ (5.1.4.5-a)  

#### 室内へ流入する吸収日射量

（参考：JIS A 2103：2014 5.4.2 計算基礎式）

室内へ流入する吸収日射熱量$I_{qi}$の計算方法を以下に示す。  

$ \begin{align} I_{qi} = \sum_{j=0}^{N_{lay}-1} I_{\alpha,j}･N_{in,j} \end{align}$ ････････････････････････････････････････････････････････････････････････････････ (5.1.4.5-b)  


In [ ]:
def Secondary_Heat_Transfer(L):

    sum_R = 0.
    R_in = np.zeros(len(L.qin_ia))
    for j in range(len(L.qin_ia)):
        sum_R += L.qin_rglz[2 * j] + L.qin_rglz[2 * j + 1]
        R_in[j] = sum_R -  L.qin_rglz[2 * j + 1] / 2.

    R_tot = sum_R + L.qin_rglz[2 * len(L.qin_ia)]
    N_in = np.zeros(len(L.qin_ia))
    r_qin = 0.
    for j in range(len(L.qin_ia)):
        N_in[j] = R_in[j] / R_tot
        r_qin += L.qin_ia[j] * N_in[j]
        
    return r_qin

### 計算例

In [ ]:
qin_ia =[109.03733774, 84.53584395, 63.94106107]
qin_rglz = [0.04178292, 0.003, 0.1761019, 0.003, 0.16618873, 0.003, 0.12007248]

qin_in = qin_input(qin_ia, qin_rglz)
I_qi = Secondary_Heat_Transfer(qin_in)

print (I_qi)

### 5.1.4.6. 日射熱取得率

説明：開口面入射日射量と開口面透過日射量及び室内側へ流入する吸収日射量からグレージング複合体の日射熱取得率を計算する。

### 入力値  

$ I_{in} $ ：開口面入射日射量 [MJ/(m<sup>2</sup>･h)]  
$ I_{\tau} $ ：開口面透過日射量 [MJ/(m<sup>2</sup>･h)]  
$ I_{qi} $ ：室内側へ流入する吸収日射量 [MJ/(m<sup>2</sup>･h)]  

### 出力値  

$ \eta_{glz} $ ：グレージングの日射熱取得率  

### 計算方法  

#### データ構造と値の範囲  
$ I_{T} $・$ I_{\tau} $・$ I_{qi} $の組み合わせを入力条件として1つの変数（タプル）で表し"ghg_input"とする。  


In [ ]:
ghg_input = namedtuple('ghg_input','ghg_Iin ghg_Itau ghg_Iqi')
#ghg_Iin：開口面入射日射量 [MJ/(m2･h)] 
#ghg_Itau：開口面透過日射量 [MJ/(m2･h)] 
#ghg_Iqi：室内側へ流入する吸収日射量 [MJ/(m2･h)] 

#### 時別の日射熱取得率

時別の日射熱取得率$\eta_{glz}$の計算方法を以下に示す。  

$ \begin{align} \eta_{glz} = \frac {I_{\tau} + I_{qi}}{I_{T}} \end{align}$ ････････････････････････････････････････････････････････････････････ (5.1.4.6-a)  


In [ ]:
def hourly_gvalue_glazing(L):
    
    if L.ghg_Iin > 0.:
        r_ghg = (L.ghg_Itau + L.ghg_Iqi)/ L.ghg_Iin
    else:
        r_ghg = 0.
        
    return r_ghg

### 計算例

In [ ]:
ghg_in = ghg_input(804.5053722499999 * 3600. / 10. ** 6., 418.407541795 * 3600. / 10. ** 6., 94.625249426 * 3600. / 10. ** 6.)
ghg = hourly_gvalue_glazing(ghg_in)

print(ghg)

### 5.2. フレームの日射熱取得率

説明：フレームの熱貫流率、屋外側表面日射吸収率、屋外側展開面積比からフレームの日射熱取得率を計算する。

### 入力値  

$ \alpha_{fr,ex} $ ：フレームの屋外側表面日射吸収率  
$ U_{fr} $ ：フレームの熱貫流率 [W/(m<sup>2</sup>)･K)]   
$ F_{de,ex} $ ：フレームの屋外側展開面積比   
$ h_{fr,ex} $ ：フレームの屋外側表面熱伝達率 [W/(m<sup>2</sup>)･K)]   

### 出力値  

$ \eta_{fr} $ ：フレームの日射熱取得率  

### 計算方法  

#### データ構造と値の範囲  

im・$ \alpha_{fr,ex} $(または$ \alpha_{fr,ex} $の表ID)・$ U_{fr} $(または$ U_{fr} $の表ID)・$ F_{de,ex} $(または$ F_{de,ex} $の表ID)・$ h_{fr,ex} $(または$ h_{fr,ex} $の表ID) の組み合わせを入力条件として1つの変数（タプル）で表し"ghf_input"とする。  

In [ ]:
ghf_input = namedtuple('ghf_input','ghf_im ghf_ap ghf_Up ghf_Fp ghf_hp')
#ghf_im：データ取得方法（0：表参照入力、1：直接入力）
#ghf_ap：ghf_imが0の場合は、屋外側表面日射吸収率のID（0：省エネ基準住宅、1：省エネ基準非住宅、2～10：JIS A 2103の材質・色別）
#        ghf_imが1の場合は、屋外側表面日射吸収率
#ghf_Up：ghf_imが0の場合は、熱貫流率のID（0：JIS A 2103の樹脂、1：JIS A 2103のアルミ樹脂複合、2：JIS A 2103のアルミ）
#        ghf_imが1の場合は、熱貫流率[W/(m2･K)]
#ghf_Fp：ghf_imが0の場合は、屋外側展開面積比のID（0：省エネ基準住宅、1：省エネ基準非住宅、2～5：JIS A 2103の開閉形式別）
#        ghf_imが1の場合は、屋外側展開面積比
#ghf_hp：ghf_imが0の場合は、屋外側表面熱伝達率のID（0：省エネ基準住宅、1：省エネ基準非住宅、2：JIS A 2103の夏期、3：JIS A 2103の冬期）
#        ghf_imが1の場合は、屋外側表面熱伝達率[W/(m2･K)]

#### フレームの日射熱取得率の計算方法  

（参考：JIS A 2103：2014 5.3 フレームの日射熱取得率）

フレームの日射熱取得率$\eta_{fr}$の計算方法を以下に示す。  

$ \begin{align} \eta_{fr} = \frac {\alpha_{fr,ex}・U_{fr}}{ F_{de,ex}・h_{fr,ex}} \end{align}$ ･････････････････････････････････････････････････････････････ (5.2-a)  

ここで、  
$ \alpha_{fr,ex} $ ：フレームの屋外側表面日射吸収率  
$ U_{fr} $ ：フレームの熱貫流率 [W/(m<sup>2</sup>･K)]  
$ F_{de,ex} $ ：フレームの屋外側展開面積比  
$ h_{fr,ex} $ ：フレームの屋外側表面熱伝達率 [W/(m<sup>2</sup>･K)]  

$ \begin{align} F_{de,ex} = \frac {A_{de,ex}}{A_{fr,in}} \end{align}$ ･･････････････････････････････････････････････････････････････････ (5.2-b)  

ここで、  
$ A_{de,ex} $ ：屋外側フレーム展開面積 (m<sup>2</sup>)  
$ A_{fr,in} $ ：室内側投影フレーム面積 (m<sup>2</sup>)  

である。 

$ \alpha_{fr,ex} $の代表的な値を以下に示す（参考：平成25年省エネルギー基準に準拠した算定・判断の方法及び解説 II 住宅 第四部第三節及びJIS A 2103：2014 表A.1）。

| ID   | 規格・基準         | 材質         | 色           | 屋外側表面日射吸収率$\alpha_{fr,ex}$ |
| ---: | :---               | :---         | :---         | ---:                                 |
| 0    | 省エネ基準(住宅)   |―            | ―           | 0.80                                 |
| 1    | 省エネ基準(非住宅) |―            | ―           | 0.80                                 |
| 2    | JIS A 2103         | 樹脂         | ブラック     | 0.69                                 |
| 3    |                    |              | ブラウン     | 0.68                                 |
| 4    |                    |              | グレイ       | 0.67                                 |
| 5    |                    |              | ホワイト     | 0.22                                 |
| 6    |                    | アルミニウム | ブラック     | 0.87                                 |
| 7    |                    |              | ブラウン     | 0.76                                 |
| 8    |                    |              | グレイ       | 0.47                                 |
| 9    |                    |              | シルバー     | 0.26                                 |
| 10   |                    |              | ホワイト     | 0.24                                 |

$ U_{fr} $の代表的な値を以下に示す（参考：JIS A 2103：2014 表A.3）。  

| ID   | 規格・基準         | 材質           | 熱貫流率$U_{fr}$ [W/(m<sup>2</sup>･K)] |
| ---: | :---               | :---           | ---:                                   |
| 0    | JIS A 2103         | 樹脂           | 2.2                                    |
| 1    |                    | アルミ樹脂複合 | 4.7                                    |
| 2    |                    | アルミニウム   | 6.6                                    |

$ F_{de,ex} $の代表的な値を以下に示す（参考：平成25年省エネルギー基準に準拠した算定・判断の方法及び解説 II 住宅 第四部第三節及びJIS A 2103：2014 表A.2）。  

| ID   | 規格・基準         | 開閉形式 | 屋外側展開面積比$F_{de,ex}$ |
| ---: | :---               | :---     | ---:                        |
| 0    | 省エネ基準(住宅)   | ―       | 1.0                         |
| 1    | 省エネ基準(非住宅) | ―       | 1.0                         |
| 2    | JIS A 2103         | FIX      | 1.9                         |
| 3    |                    | 開き     | 1.5                         |
| 4    |                    | 引違い   | 2.1                         |
| 5    |                    | 上げ下げ | 1.6                         |

$ h_{fr,ex} $の代表的な物性値を以下に示す（参考：平成25年省エネルギー基準に準拠した算定・判断の方法及び解説 II 住宅 第四部第三節及びJIS A 2103：2014 5.3）。  

| ID   | 規格・基準         | 季節 | 屋外側表面熱伝達率$h_{fr,ex}$ [W/(m<sup>2</sup>･K)] |
| ---: | :---               | :--- | ---:                                                |
| 0    | 省エネ基準(住宅)   | ―   | 23                                                  |
| 1    | 省エネ基準(非住宅) | ―   | 25 (=1/0.04)                                        |
| 2    | JIS A 2103         | 夏期 | 14                                                  |
| 3    |                    | 冬期 | 24                                                  |

In [ ]:
def hourly_gvalue_frame(L):
    
    if L.ghf_im == 0:     #表参照入力
        ghf_a = frame_prop(L)[0]
        ghf_U = frame_prop(L)[1]
        ghf_F = frame_prop(L)[2]
        ghf_h = frame_prop(L)[3]
    elif L.ghf_im == 1:   #直接入力
        ghf_a = L.ghf_ap
        ghf_U = L.ghf_Up
        ghf_F = L.ghf_Fp
        ghf_h = L.ghf_hp
    
    return ghf_a * ghf_U / (ghf_F * ghf_h)

def frame_prop(L):
    
    r_ghf_a = [0.8, 0.8, 0.69, 0.68, 0.67, 0.22, 0.87, 0.76, 0.47, 0.26, 0.24]
    r_ghf_U = [2.2, 4.7, 6.6]
    r_ghf_F = [1., 1., 1.9, 1.5, 2.1, 1.6]
    r_ghf_h = [23., 25., 14., 24.]
    
    return r_ghf_a[L.ghf_ap], r_ghf_U[L.ghf_Up], r_ghf_F[L.ghf_Fp], r_ghf_h[L.ghf_hp]

### 計算例

In [ ]:
ghf_in = ghf_input(0, 0, 0, 0, 0)

ghf = hourly_gvalue_frame(ghf_in)

print(ghf)

### 5.3. 全体計算

### 5.3.1. 時刻別

説明：グレージング複合体の日射熱取得率、フレームの日射熱取得率、グレージング面積比又はグレージング面積及びフレーム面積から開口部の日射熱取得率を計算する。

### 入力値  

$ \eta_{glz} $ ：グレージング複合体の日射熱取得率  
$ \eta_{fr} $ ：フレームの日射熱取得率   
$ F_{Ag} $ ：グレージング面積比   
または、  
$ A_{glz} $ ：グレージング面積 (m<sup>2</sup>)   
$ A_{fr} $ ：フレーム面積 (m<sup>2</sup>)   

### 出力値  

$ \eta_{w} $ ：開口部の日射熱取得率  

### 計算方法  

#### データ構造と値の範囲  

$ \eta_{glz} $・$ \eta_{fr} $・im・fm・$ A_{glz} $・$ A_{fr} $の組み合わせを入力条件として1つの変数（タプル）で表し"ghw_input"とする。  

In [ ]:
ghw_input = namedtuple('ghw_input','ghw_gg ghw_gf ghw_im ghw_fm ghw_Ag ghw_Af')
#ghw_gg：グレージング複合体の日射熱取得率
#ghw_gf：フレームの日射熱取得率
#ghw_im：グレージング面積比のデータ取得方法（0：表参照入力、1：直接入力）
#ghw_fm：フレーム材質のID（0：木製建具又は樹脂製建具、
#                          1：木と金属の複合材料製建具、樹脂と金属の複合材料製建具、金属製熱遮断構造建具又は金属製建具）
#                          （※ghw_imが1の場合は値を無視）
#ghw_Ag：グレージング面積 (m2)（※ghw_imが0の場合は値を無視）
#ghw_Af：フレーム面積 (m2)（※ghw_imが0の場合は値を無視）

#### 開口部の日射熱取得率の計算方法  

開口部の日射熱取得率$\eta_{w}$の計算方法を以下に示す。  

$ \begin{align} \eta_{w} = F_{Ag}・\eta_{glz} + (1 - F_{Ag})・\eta_{fr} \end{align}$ ･････････････････････････････････････ (5.3.1-a)  

ここで、  
$ \eta_{glz} $ ：グレージング複合体の日射熱取得率  
$ \eta_{fr} $ ：フレームの日射熱取得率  
$ F_{Ag} $ ：グレージング面積比  

$ \begin{align} F_{Ag} = \frac {A_{glz}}{A_{glz} + A_{fr}} \end{align}$ ･･･････････････････････････････････････････････････ (5.3.1-b)  

ここで、  
$ A_{glz} $ ：グレージング面積 (m<sup>2</sup>)  
$ A_{fr} $ ：フレーム面積 (m<sup>2</sup>)  

である。 

$ F_{Ag} $の代表的な物性値を以下に示す（参考：平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）Ver.2.1 算定方法 3.4　日射熱取得率）。  

| ID   | 材質                                                                                     | グレージング面積比$F_{Ag}$ |
| ---: | :---                                                                                     | ---:                       |
| 0    | 木製建具又は樹脂製建具                                                                   | 0.72                       |
| 1    | 木と金属の複合材料製建具、樹脂と金属の複合材料製建具、金属製熱遮断構造建具又は金属製建具 | 0.80                       |

In [ ]:
def hourly_gvalue_window(L):
    
    if L.ghw_im == 0:     #表参照入力
        F_Ag = glazing_area_ratio(L)
    elif L.ghw_im == 1:   #直接入力    
        F_Ag = L.ghw_Ag / (L.ghw_Ag + L.ghw_Af)
    
    return F_Ag * L.ghw_gg + (1. - F_Ag) * L.ghw_gf

def glazing_area_ratio(L):
    
    r_F_Ag = [0.8, 0.72]
    
    return r_F_Ag[L.ghw_fm]

### 計算例

In [ ]:
ghw_in = ghw_input(0.65, 0.05, 0, 0, 0, 0)

ghw = hourly_gvalue_window(ghw_in)

print(ghw)

### 全体計算例（グレージング複合体のみ）

In [ ]:
#入力値
#気象データ
location = '3639999'               #東京ID
year = 1995                        #西暦年
lat = 35. + 41.2 / 60.             #計算対象地点の緯度 (°)  
lon = 139. + 45.9 / 60.            #計算対象地点の経度 (°)  
Bw = 90.                           #水平面に対する斜面（開口面）の傾斜角 (°) 
Aw = 0.                            #斜面（開口面）の方位角 (°)
rho = 0.1                          #アルベド

#グレージング
gl_type = [0, 0, 0]                #各層のグレージング種類（0：ガラス、1：ロールスクリーン、2：横型ブラインド、3：縦型ブラインド）
gl_d = [[0.003], [0.003], [0.003]] #各層の厚さ
gl_lmd = [[1.], [1.], [1.]]        #各層の熱伝導率
gl_trf = [0.815, 0.815, 0.815]     #各層の正面側の日射透過率
gl_trb = [0.815, 0.815, 0.815]     #各層の背面側の日射透過率
gl_ref = [0.072, 0.072, 0.072]     #各層の正面側の日射反射率
gl_reb = [0.072, 0.072, 0.072]     #各層の背面側の日射反射率
gl_epf = [0.837, 0.837, 0.837]     #各層の正面側の放射率
gl_epb = [0.837, 0.837, 0.837]     #各層の背面側の放射率
gl_ty = [0, 0, 0]                  #各層のガラスの種類（0：透明フロート板ガラス、1：Low-Eガラス）
gl_cf = [0, 0, 0]                  #各層の#正面側の膜の有無（0：膜なし、1：膜あり）
gl_cb = [0, 0, 0]                  #各層の#背面側の膜の有無（0：膜なし、1：膜あり）

#中空層
gp_d = [0.012, 0.012]              #各層の厚さ
gp_gas1 = [0, 0]                   #各層の気体1の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
gp_gas2 = [0, 0]                   #各層の気体2の種類flag（0：空気、1：アルゴン、2：SF6、3：クリプトン）
gp_gas1r = [1., 1.]                #各層の気体1の容積割合（0.0～1.0）
gp_dir = [0, 0]                    #熱流方向flag（0：中空層が垂直で熱流方向が水平、1：中空層が水平で熱流方向が上向き、
                                   #              2：中空層が45度で熱流方向が上向き）

#天空分割数
Nd_lat = 90                        #緯度方向
Nd_lon = 180                       #経度方向

#地表面反射分割数
Nr_lat = 90                        #緯度方向
Nr_lon = 180                       #経度方向

#
season = 0 #夏期：0、冬期：1-->暖冷房期間から決定すべき
if season == 0:
    ti = 25.  #室温
elif season == 1:
    ti = 20.  #室温
    
for j in range(len(gl_d)):
    if gl_type[j] == 0:            #ガラス
        gl_in[j] = glass_input(gl_trf[j], gl_trb[j], gl_ref[j], gl_reb[j], gl_ty[j], gl_cf[j], gl_cb[j])
    elif gl_type[j] == 1:          #ロールスクリーン
        rl_in[j] = role_input(gl_trf[j], gl_trb[j], gl_ref[j], gl_reb[j])
    
tau_d = actual_trans_diffuse(gl_in, Nd_lat, Nd_lon)
tau_r = actual_trans_reflect(gl_in, Nr_lat, Nr_lon)

abs_d = np.zeros(len(gl_d))
abs_r = np.zeros(len(gl_d))
for j in range(len(gl_d)):
    abs_d[j] = actual_abs_diffuse(gl_in, Nd_lat, Nd_lon)[j]
    abs_r[j] = actual_abs_reflect(gl_in, Nr_lat, Nr_lon)[j]

I_in_J = np.zeros(8760)
I_tau_J = np.zeros(8760)
I_qi_J = np.zeros(8760)

I_abs = np.zeros(len(gl_d))
Rglz = np.empty(2 * len(gl_d) + 1, dtype = tuple)
ghg = np.zeros(8760)

#気象データ読込
tmp = Weather.weather_input(location)[3]                                       #外気温
ig = Weather.weather_input(location)[4]                                        #水平面全天日射量

for i in range(0, 8760):
#i = 0
    sp_in = sp_input(year, int((i + 1.) / 24.) + 1., ((i + 1.) / 24. - int((i + 1.) / 24.)) * 24., lat, lon)
    sp_h = SolarPosition.solar_position(sp_in)[0]                                    #太陽高度角
    sp_A = SolarPosition.solar_position(sp_in)[1]                                    #太陽方位角

    dbd_in = dbd_input(ig[i], sp_h)
    dbd_ib = SplittingGlobal2BeamDiffuseRadiation.division_beam_diffuse(dbd_in)[0]                          #法線面直達日射量
    dbd_id = SplittingGlobal2BeamDiffuseRadiation.division_beam_diffuse(dbd_in)[1]                          #水平面天空日射量

    st_in = st_input(ig[i], dbd_ib, dbd_id, sp_h, sp_A, Bw, Aw, rho)
    st_itb = solor_tilt(st_in)[0]                                      #斜面（開口面）直達日射量
    st_itd = solor_tilt(st_in)[1]                                      #斜面（開口面）天空日射量
    st_itr = solor_tilt(st_in)[2]                                      #斜面（開口面）地表面反射日射量
    phi_b = solor_tilt(st_in)[3]                                       #直達日射の入射角

#単位変換(J2W)
    I_tb = transform_unit_joule2watt(st_itb)                           #斜面（開口面）直達日射量
    I_td = transform_unit_joule2watt(st_itd)                           #斜面（開口面）天空日射量
    I_tr = transform_unit_joule2watt(st_itr)                           #斜面（開口面）地表面反射日射量

#開口面入射日射量
    ita_in = ita_input(I_tb, I_td, I_tr)
    I_in = incident_sol(ita_in)
    I_in_J[i] = transform_unit_watt2joule(I_in)
    
#開口面透過日射量
    tau_b = actual_trans_beam(gl_in, phi_b)
    tta_in = tta_input(tau_b, tau_d, tau_r)
    I_tau = transmit_sol(ita_in, tta_in)
    I_tau_J[i] = transform_unit_watt2joule(I_tau)
    
#層jの吸収日射量
    abs_b = np.zeros(len(gl_d))
    for j in range(len(gl_d)):
        abs_b[j] = actual_abs_beam(gl_in, phi_b)[j]
        
        ata_in = ata_input(abs_b[j], abs_d[j], abs_r[j])
        I_abs[j] = absorb_sol(ita_in, ata_in)   

#熱平衡
    glp_in = glp_input(gl_d, gl_lmd, I_abs[0:len(gl_d)], gl_epf, gl_epb)
    gpp_in = gpp_input(gp_d, gp_gas1, gp_gas2, gp_gas1r, gp_dir)    
    wp_in = wp_input(season, tmp[i], ti)    
    
    hbr = Heat_balance(glp_in, gpp_in, wp_in)
    
    Rglz = hbr[1][0 : 2 * len(gl_d) + 1]

#室内側へ流入する吸収日射量
    qin_in = qin_input(I_abs[0:len(gl_d)], Rglz)
    I_qi = Secondary_Heat_Transfer(qin_in)   
    I_qi_J[i] = transform_unit_watt2joule(I_qi)
    
#時別の日射熱取得率   
    ghg_in = ghg_input(I_in_J[i], I_tau_J[i], I_qi_J[i])
    ghg[i] = hourly_gvalue_glazing(ghg_in)
      
    #print (i, I_in_J[i], I_tau_J[i], I_qi_J[i], ghg[i])
    #print (i)
    
np.savetxt(location + '_out_I_in.csv', I_in_J, fmt='%.10f', delimiter=',')
np.savetxt(location + '_out_I_tau.csv', I_tau_J, fmt='%.10f', delimiter=',')
np.savetxt(location + '_out_I_qi.csv', I_qi_J, fmt='%.10f', delimiter=',')
np.savetxt(location + '_out_ghg.csv', ghg, fmt='%.10f', delimiter=',')

### 5.3.2. 期間平均

### 5.3.2.1. 暖房期間

### 5.3.2.2. 冷房期間

## 6. 窓の熱貫流率の計算